In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, mne, json, h5py, pymatreader, re, time, gc;
import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mpr
import matplotlib as mpl
import scipy.signal as sig
import pandas as pd 
import utils_postprocess_HPC as postp

data_dir = gv.data_dir
from os.path import join as pjoin

light_only = 1
#light_only = 0
ndaysBefore = None

from dateutil import parser
# start_time = parser.parse("6 Sept 2021 19:05:15")
# end_time = parser.parse("8 Sept 2021 21:21:45")
#start_time = parser.parse("26 Sept 2021 01:00:15")
#start_time = parser.parse("27 April 2022 16:30:15")
start_time = parser.parse("2 Sept 2022 09:00:00")
#end_time = parser.parse("30 Oct 2021 21:21:45")
end_time = parser.parse("30 Oct 2029 21:21:45")

ndaysBefore = None
#subdir = 'nointerp'
#subdir = 'nofeatsel'
subdir = 'joint_best_LFP_wholectx'
lookup_dir = pjoin(gv.data_dir,subdir)
recent = postp.listRecent(days=ndaysBefore, lookup_dir= lookup_dir,
                          start_time=start_time,
                                   end_time=end_time)
print(len(recent))
rawnames = []
rawname_regex = '([S0-9,]+)'
re1 = re.compile('_\!_'+rawname_regex+'_.*')
re2 = re.compile('_'+rawname_regex+'_.*')
for lf in recent:
    st = 0
    if light_only:
        if not lf.startswith('_!'):
            continue    
    if light_only:
        r = re.match(re1,lf)
    else:
        r = re.match(re2,lf)
    if r is None:
        print('None ',lf)
        continue
    cr = r.groups()[0]
    if cr not in rawnames:
        rawnames += [ cr ]
rawnames = list(sorted(set(rawnames)))

#a0 = re.findall('_\!_'+rawname_regex+'_.*',lf)
#a1 = re.match('_\!_'+rawname_regex+'_.*',lf)
#print(a0,a1.groups())#

import utils_postprocess_HPC as postp
prefixes = postp.listRecentPrefixes(days = ndaysBefore, light_only=light_only, 
                                    lookup_dir= lookup_dir, 
                                    custom_rawname_regex = rawname_regex, 
                                    start_time=start_time,
                                   end_time=end_time)

print(rawnames)
display(prefixes)

In [ ]:
#to_remove = ['onlyH_act_only50', 'onlyH_act_only49']
# to_remove = ['onlyH_act_only50']
# for pref in to_remove:
#     prefixes.remove(pref)

In [ ]:
from utils_postprocess import printSizeInfo
#for i,tpl in enumerate(tpll):
    #print(i); printSizeInfo(tpl[-1])
printSizeInfo(tpll[4][-1],depthcur=0,depthleft=5 , minsize=1e-10)

In [ ]:
#('S01_off', 'modLFP') {'trem_L': '19.2%=64.3s', 'notrem_L': '32.3%=108.0s', 'hold_L': '24.8%=83.0s', 'move_L': '23.6%=79.0s'}

In [ ]:
from utils_postprocess_HPC import checkPrefixCollectionConsistencty
#for grouping_to_check in ['merge_nothing']: #, 'merge_movements']:
for grouping_to_check,it_to_check in [ ('merge_all_not_trem','basic'), ('merge_movements','basic'),
                                      ('merge_nothing','basic'), ('merge_nothing','trem_vs_quiet') ]:
    print('   ',grouping_to_check,it_to_check)
    r = checkPrefixCollectionConsistencty(subdir,prefixes,start_time, end_time,  
                                          grouping_to_check, it_to_check,
                                          use_main_LFP_chan=1, light_only=1, 
                                         prefixes_ignore  = [], preloaded=None)
    missing, preloaded = r
    print('missing=', missing)
import gc; gc.collect()

In [ ]:
# ff = preloaded['S02_on']['onlyH_act_only15']['merge_nothing']['basic']['_fname_full']
# ff,os.path.exists(ff)

# f = np.load(ff,allow_pickle=1)
# rt = f['results_light'][()]

In [ ]:
#del f,rt,ff

# Load

In [ ]:
sources_type = 'parcel_aal'  # or ''
load = False # whether I want to do actual load or just collect filenames (for deferred loading)
r = postp.collectPerformanceInfo3(None,prefixes, nraws_used='[0-9]+',                                                                                                    
          sources_type = sources_type, 
           printFilenames=1,                                 
            ndays_before=ndaysBefore,
            use_main_LFP_chan=1,
             subdir=subdir, remove_large_items = 1,
             list_only=0, allow_multi_fn_same_prefix=0,
             use_light_files = light_only, rawname_regex_full=1,
             start_time=start_time,
               end_time=end_time, load=load)

#nraws_used='(10,12,20,24)'
#output_per_raw,Ximp_per_raw,gis_per_raw = r
output_per_raw,_,_ = r
print('len(output_per_raw) =', len(output_per_raw))
import gc; gc.collect()

if load:
    np.savez(pjoin(gv.data_dir,subdir,'gathered.npz'), output_per_raw=output_per_raw )
import gc; gc.collect()

#Audio(filename=sound_file, autoplay=True)
import utils_postprocess as pp
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)

z0 = [tpl[:-1] for tpl in tpll]
#rns_ord, prefs_ord, grp_ord, it_ord = list (zip(*z0  ) )
tpll_reshaped = np.array( list (zip(*z0  ) ) )

In [ ]:
from datetime import datetime
from pathlib import Path
for tpl in tpll:
    moc = tpl[-1]
    fn = Path(moc['filename_full'] ).name
    if fn.startswith('_!'):
        continue
    else:
        print(fn , datetime.fromtimestamp(moc['mod_time'] ) )

In [ ]:
test_rn = 'S01,S04,S05'
test_rn_comb = 'S01,S04,S05_off,on'
if test_rn in rawnames:
    del rawnames[rawnames.index(test_rn)]
if test_rn_comb in output_per_raw:
    del output_per_raw[test_rn_comb]
    
# rns_inc_S05 = ['S01,S02,S04,S05,S07_on', 'S01,S02,S03,S04,S05,S07_off']
# for rn in rns_inc_S05:
#     if rn in rawnames:
#         del rawnames[rawnames.index(rn)]
#     if rn in output_per_raw:
#         del output_per_raw[rn]
rawnames, output_per_raw.keys()

In [ ]:
from utils_postprocess_HPC import getOutputSetInfo
getOutputSetInfo(output_per_raw)

### Check runstrings gens ids present and times distribution

In [ ]:
import utils_postprocess as pp
from datetime import datetime
from utils_postprocess_HPC import loadRunCorresp
CIDs,cretimes = [],[]
mod_times = []
nloaded = 0
for tpl in tpll:
    moc = tpl[-1]
    corresp,all_info = loadRunCorresp(moc)
    if moc['loaded']:
        if 'cmd' not in moc:
            print(f'Cmd is not in {tpl[:-1]}')
            break
        runCID = dict( moc['cmd'][0] ).get('--runCID', None) 
        moc['runstrings_creation_time'] = all_info['date_created']
        moc['runCID'] = runCID
        CIDs += [runCID]
        cretimes += [all_info['date_created']]

        #mod_time = os.stat( moc['filename_full'] ).st_mtime
        mod_time = moc['mod_time']
        dt = datetime.fromtimestamp(mod_time)
        moc['mod_time_dt'] = dt
        nloaded += 1
    
    mod_times += [moc['mod_time']] # timestamps only

bv,bcoord,_ = plt.hist(mod_times)
mn = datetime.fromtimestamp ( np.min(bcoord) )
mx = datetime.fromtimestamp ( np.max(bcoord) )
delta = mx-mn
print(f'Time spread = {delta}, time min = {mn}, time max = {mx}')

In [ ]:
if not nloaded:
    print('Nothing is loaded :(')
CIDs = list( map(int,CIDs) ) 
CIDs_sorted = list( sorted( set( map(int,CIDs) ) ) )
for CID in CIDs_sorted:
    inds = np.where( np.array(CIDs) == CID )[0]
    times = [tpll[i][-1]['fname_mod_time'] for i in inds]
    times = list( sorted(times) )
    print('CID =  ',CID, 'len(inds)=',len(inds), times[0], times[-1] )
CID_most_recent = list( sorted( map(int,CIDs) ) )[-1]
print('most recent CID=', CID_most_recent )

In [ ]:
if not nloaded:
    print('Nothing is loaded :(')
# load large file with gathered
import utils_postprocess as pp
load_precollected = 0
if load_precollected:
    output_per_raw = np.load(pjoin(gv.data_dir,subdir,'gathered.npz'),allow_pickle=1)
    output_per_raw = output_per_raw['output_per_raw'][()]
    print('len(output_per_raw) =', len(output_per_raw))
    tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)

    z0 = [tpl[:-1] for tpl in tpll]
    #rns_ord, prefs_ord, grp_ord, it_ord = list (zip(*z0  ) )
    tpll_reshaped = np.array( list (zip(*z0  ) ) )

### Check completness of loaded

In [ ]:
from utils_postprocess_HPC import checkTupleListTableCompleteness
b,missing = checkTupleListTableCompleteness(tpll_reshaped)

outputs_filtered = postp.filterOutputs(output_per_raw, rns=rawnames, 
                    prefs=prefixes, grps = ['merge_nothing'] )
b,missing =  checkTupleListTableCompleteness(outputs_filtered)
print(missing)

## Tremor and other beh states durations

In [ ]:
# TODO NEXT -- try to compare merge nothing with other groupings

In [ ]:
if not nloaded:
    print('Nothing is loaded :(')
grp = 'merge_nothing'
#pref_print = 'modLFP'
#ref_print = 'onlyH_act_LFPand_only13'
pref_print = 'onlyH_actLL'
print('pref_print', pref_print)
r = {}
for tpl in sorted(tpll, key= lambda x: rawnames.index(x[0]) * 1000 + prefixes.index(x[1]) ):
    if tpl[2] != grp or tpl[1] != pref_print:
        continue
    cts = tpl[-1]['counts']
    tot = np.sum( list(cts.values()) )
    cts2 = cts.copy()
    s = ''
    for k,v in cts.items():
        k2 = k[:-2]
        #k2 = k
        stmp = f'{v / tot * 100:4.1f}%={v/32:5.1f}s'
        cts2[k2] = stmp
        s += f', {k2}: {stmp}'
    #print(tpl[:2],  cts2)
    print(f'{tpl[0]:7} {s[1:]}')
    #print(tpl[:2],  cts2['trem_L'])
    #k = ','.join( tpl[:-1] )
    k = tpl[:-1]
    r[k] = {}
    r[k]['counts'] = cts
    r[k]['infostr'] = s
    r[k]['total'] = tot / 32
    r[k]['bacc'] = tpl[-1]['XGB_analysis_versions']['all_present_features']['perf_dict']['perf_aver']['balanced_accuracy']
    
if len(r):
    # import json
    # fn_full  = pjoin(gv.data_dir,subdir,'beh_states_durations.json')
    # with open(fn_full,'w') as f:
    #     json.dump(r,f)
    fn_full  = pjoin(gv.data_dir,subdir,
        f'beh_states_durations_CID{CID_most_recent}.npz')
    np.savez(fn_full, r)
    print('beh state duration saved to ',fn_full)
else:
    print('zero len!')

In [ ]:
#r

In [ ]:
r

In [ ]:
# REGEX =  '_!_([,0123456789S_defhlmnorstv]+)_parcel_aal_grp10-0_([a-zA-Z0-9_,:]+)_ML_nr[0-9]{2,4}_.*'#'[0-9]+' +\
# #'chs_nfeats[0-9]+_pcadim[0-9]+_skip[0-9]+_wsz[0-9]+_mainLFP__\(([a-zA-Z0-9_&]+),([a-zA-Z0-9_&]+)\).npz'
# #rawname_regex = '([S0-9]+_[a-z]+)'
# a1 = re.match(REGEX,lf)
# a1

# Starting plotting

### prepare table info (long)

In [ ]:
#%debug
# mult_clf_output = tpll[0][-1]
# all_thrs = mult_clf_output['feat_variance_q_thr'][-1:]
# #thr0,thr1,thr2='0.87','0.92','0.99'
# all_LDA =  []
# all_XGB = ['all_present_features'] #,'after_VF_threshold']
# for thr_cur in all_thrs:
#     all_XGB += [ f'best_PCA-derived_features_{thr_cur}']

# perf_to_use_list = []
# for v in all_XGB[1:]:
#     perf_to_use_list += [('XGB',all_XGB[0],v,'across_medcond')]
    #perf_to_use_list += [('XGB',all_XGB[0],v,'across_subj')]
    
#perf_to_use_list = [('XGB',all_XGB[0], f'best_PCA-derived_features_{all_thrs[-1]}','across_subj')]

perf_to_use_list = [('XGB','all_present_features', f'best_PCA-derived_features_0.99','across_subj')]

###########################################
display(perf_to_use_list)

# to_show = [('allsep','merge_nothing','basic'), ('trem_vs_all','merge_all_not_trem','basic'),
#         ('trem_vs_2class','merge_movements','basic')]
#to_show = [('trem_vs_mvt','merge_movements','trem_vs_hold&move'),
#           ('trem_vs_all','merge_all_not_trem','basic') ]
#             ('trem_vs_2class','merge_movements','basic'),
#to_show = [('trem_vs_all','merge_all_not_trem','basic') ]
to_show = []
to_show += [('allsep','merge_nothing','basic') ]
#to_show += [('trem_vs_2class','merge_movements','basic')]
#to_show += [('trem_vs_all','merge_all_not_trem','basic')]
to_show += [('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]
# to_show += [('trem_vs_quiet','merge_all_not_trem','trem_vs_quiet') ]
#             ('trem_vs_2class','merge_movements','basic'),

#%debug
# warnings.simplefilter('error')
df, table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(output_per_raw, prefixes=prefixes, 
    perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True)

gc.collect()

df_all = df

In [ ]:
tpll[0][-1]['perfs_XGB'].keys()

In [ ]:
df.columns

In [ ]:
df['sens_add']

### Compare main stats with other feature sets

In [ ]:
grps,its = set(df['grouping']), set(df['interval_set'])
grps,its

#### Change df a bit

In [ ]:
from utils_postprocess_HPC import getTremorDetPerf,getMocFromRow
def addTremorPerf(row):
    grp,it = row['grouping'],row['interval_set']
    moc = getMocFromRow(row, output_per_raw)
    r = getTremorDetPerf(moc,grp,it) 
    return r
def addTremorPerfAcross(row):
    grp,it = row['grouping'],row['interval_set']
    moc = getMocFromRow(row, output_per_raw)
    r = getTremorDetPerf(moc,grp,it,across='subj') 
    return r

df['tremor_det_perf'] = df.apply(addTremorPerf,axis=1)
df['tremor_det_perf_across'] = df.apply(addTremorPerfAcross,axis=1)
df.reset_index()

df['num'] = pd.to_numeric(df['num'])
df['num'] = df['num'].map(lambda x: int(x) if ( not (np.isnan(x) or (x is None) ) ) else 0    )
df['numpts'] = pd.to_numeric(df['numpts'])
df['sens'] = pd.to_numeric(df['sens'])
df['spec'] = pd.to_numeric(df['spec'])

####################
import globvars as gv
fname = pjoin(gv.data_dir,subdir,'df_collected.pkl')
df.to_pickle(fname)
################

In [ ]:
# results_cur['runCID'] = runCID
# results_cur['runstring_ind'] =  runstring_ind
# results_cur['SLURM_job_id'] =  SLURM_job_id


In [ ]:
df.columns

#### Create sub df

In [ ]:
dfsub

In [ ]:
grp.count()

In [ ]:
use_prefix = False
#dfsub = df[['rawname','prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled']]
dfsub = df[['rawname','grouping','interval_set',
            'prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled','bacc_add','sens','spec',
           'tremor_det_perf','tremor_det_perf_across']]

# prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8','onlyH_mob_subskip8','onlyH_compl_subskip8','allb_beta_noH_subskip8','allb_gamma_noH_subskip8','allb_tremor_noH_subskip8',
#         'LFPrel_noself_subskip8','modSrc_self_subskip8','modSrc_subskip8','modLFP_subskip8']

if use_prefix:
    prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8',
                        'onlyH_mob_subskip8','onlyH_compl_subskip8','onlyH_modLFP_subskip8']
    prefix_role = 'prefix'
else:
    pref_templs_to_compare = ['onlyH_subskip8%%','onlyH_act_subskip8%%',
#                        'onlyH_mob_subskip8%%','onlyH_compl_subskip8%%',
                             'onlyH_modLFP_subskip8%%']
    prefix_role = 'prefix_templ'

dfsub2 = dfsub[ dfsub[prefix_role].isin(pref_templs_to_compare) ]

#dfsub2 = dfsub2[ (dfsub2['grouping'] == grp) & (dfsub2['interval_set'] == it) ]
grp =  dfsub2.groupby( by=[prefix_role,'grouping','interval_set'] )

assert tuple( set( list( grp.count()['rawname'] ) ) )  == tuple([2])
#grp.count()

#print ( len( dfsub2.groupby('prefix') ) )
dfmn = grp.mean().reset_index()
dfstd = grp.std().reset_index()

rename_dict = dict( zip( list( dfstd.columns), [col + '_std' for col in  dfstd.columns  ] ) )
del rename_dict[prefix_role]
del rename_dict['grouping']
del rename_dict['interval_set']
dfstd = dfstd.rename(columns=rename_dict)

rename_dict = dict( zip( list( dfmn.columns), [col + '_mean' for col in  dfmn.columns  ] ) )
del rename_dict[prefix_role]
del rename_dict['grouping']
del rename_dict['interval_set']
dfmn = dfmn.rename(columns=rename_dict)

dfstat = dfmn
for col in dfstd.columns:
    if col in ['prefix_templ','prefix','num_std','grouping','interval_set']:
        continue
    dfstat[col] = dfstd[col]
cols_desired = [prefix_role, 'grouping','interval_set','bacc_mean','bacc_std',
                      'bacc_add_mean','bacc_add_std',                      
                'tremor_det_perf_mean','tremor_det_perf_std', 
                'tremor_det_perf_across_mean','tremor_det_perf_across_std',
                'num_mean']
dfstat_short = dfstat[cols_desired]

display( dfstat_short )

#### print nice all info

In [ ]:
show_only_act = 0
if show_only_act:
    if prefix_role == 'prefix':
        good = 'onlyH_act_subskip8'
    else:
        good = 'onlyH_act_subskip8%%'
    dfstat_short_nice = dfstat_short[dfstat_short[prefix_role] == good].copy()    
else: # all Hjorth
    dfstat_short_nice = dfstat_short.copy()
dfstat_short_nice = dfstat_short_nice.drop(['num_mean'], axis=1) # here (unlike in medcondsep) I want to drop num_mean anyway

rename_dict = {}
for s in dfstat_short_nice[prefix_role]:
    if prefix_role == 'prefix':
        s2 = s[:-(len('subskip8')+1)]
    else:
        s2 = s[:-(len('subskip8')+3)]
    rename_dict[s] = s2
display(rename_dict)
#dfstat_short_nice.rename(lambda x: x.replace(rename_dict[x] ) , axis='columns')
dfstat_short_nice[prefix_role].replace(rename_dict, inplace=True )

assert( len( set(dfstat_short_nice['grouping'])  )  ) == 1
print(dfstat_short_nice['grouping'])
dfstat_short_nice = dfstat_short_nice.drop(['grouping'], axis=1)


rename_dict = {'onlyH':'all Hjorth parameters' , 'onlyH_act':'Hjorth activity', 
               'onlyH_mob':'Hjorth mobility', 'onlyH_compl':'Hjorth complexity', 'onlyH_modLFP':'all Hjorth parameters of LFP'}
dfstat_short_nice[prefix_role].replace(rename_dict, inplace=True )

# rename_dict = { 'merge_all_not_trem':'all non-tremor states', 
#             'merge_movements':'voluntary movements',
#             'merge_nothing':'none'          }
# dfstat_short_nice['grouping'].replace(rename_dict, inplace=True )

rename_dict = { 'basic':'tremor,quiet,hold,grasp', 
            'trem_vs_quiet':'tremor,quiet'  }
dfstat_short_nice['interval_set'].replace(rename_dict, inplace=True )

pctf = lambda x: f'{x * 100:.0f}%'
# if not show_only_act:
#     dfstat_short_nice['num_mean'] = dfstat_short_nice['num_mean'].map(int)
    
pctize = ['bacc_mean','bacc_std','tremor_det_perf_mean','tremor_det_perf_std','bacc_add_mean','bacc_add_std',
          'tremor_det_perf_across_mean','tremor_det_perf_across_std']
for coln in pctize:
    dfstat_short_nice[coln] = dfstat_short_nice[coln].apply(pctf)
# dfstat_short_nice['performance / feature number'] = \
#     dfmn.apply(lambda row: f'{row.bacc_mean / row.num_mean:.4f}', axis=1 )

# rename column names
rename_dict = {}
rename_dict[prefix_role] = 'Feature set'
#rename_dict['grouping'] = 'behav. states merged'
rename_dict['interval_set'] = 'behav. states used'
rename_dict['bacc_mean'] = 'performance mean'
rename_dict['bacc_std'] = 'performance std'
rename_dict['bacc_add_mean'] = 'performance across subj. mean'
rename_dict['bacc_add_std'] = 'performance across subj. std'

rename_dict['tremor_det_perf_mean'] = 'tremor detection mean'
rename_dict['tremor_det_perf_std'] = 'tremor detection std'

rename_dict['tremor_det_perf_across_mean'] = 'tremor detection across subj. mean'
rename_dict['tremor_det_perf_across_std'] = 'tremor detection across subj. std'
# if not show_only_act:
#     rename_dict['num_mean'] = 'feature number'
dfstat_short_nice.rename(columns=rename_dict, inplace=True)

columns = np.array(list(dfstat_short_nice.columns) )[[0,1,2,5,6,3,4]]
columns

#dfstat_short_nice[columns]
dfstat_short_nice.style.set_properties(**{'text-align': 'left'})

#### create sub df (without grouping of raws)

In [ ]:
use_prefix = False
#dfsub = df[['rawname','prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled']]
dfsub = df[['rawname','grouping','interval_set',
            'prefix','prefix_templ', 'num','numpts','bacc','bacc_shuffled','bacc_add','sens','spec',
           'tremor_det_perf','tremor_det_perf_across']]

# prefs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8','onlyH_mob_subskip8','onlyH_compl_subskip8','allb_beta_noH_subskip8','allb_gamma_noH_subskip8','allb_tremor_noH_subskip8',
#         'LFPrel_noself_subskip8','modSrc_self_subskip8','modSrc_subskip8','modLFP_subskip8']

if use_prefix:
    pref_templs_to_compare = ['onlyH_subskip8','onlyH_act_subskip8',
                        'onlyH_mob_subskip8','onlyH_compl_subskip8','onlyH_modLFP_subskip8']
    prefix_role = 'prefix'
else:
    pref_templs_to_compare = ['onlyH_subskip8%%','onlyH_act_subskip8%%',
#                        'onlyH_mob_subskip8%%','onlyH_compl_subskip8%%',
                             'onlyH_modLFP_subskip8%%']
    prefix_role = 'prefix_templ'

dfstat = dfsub[ dfsub[prefix_role].isin(pref_templs_to_compare) ]

for col in dfstd.columns:
    if col in ['prefix_templ','prefix','num_std','grouping','interval_set']:
        continue
    dfstat[col] = dfstd[col]
cols_desired = ['rawname', prefix_role, 'grouping','interval_set','bacc',
                      'bacc_add',                      
                'tremor_det_perf',
                'tremor_det_perf_across',
                'num']
dfstat_short = dfstat[cols_desired]

display( dfstat_short )

#### print nice all info (w/o gropuing across rawnames)

In [ ]:
import gc; gc.collect()

In [ ]:
show_only_act = 0
if show_only_act:
    if prefix_role == 'prefix':
        good = 'onlyH_act_subskip8'
    else:
        good = 'onlyH_act_subskip8%%'
    dfstat_short_nice = dfstat_short[dfstat_short[prefix_role] == good].copy()    
else: # all Hjorth
    dfstat_short_nice = dfstat_short.copy()
dfstat_short_nice = dfstat_short_nice.drop(['num'], axis=1) # here (unlike in medcondsep) I want to drop num_mean anyway

rename_dict = {}
for s in dfstat_short_nice[prefix_role]:
    if prefix_role == 'prefix':
        s2 = s[:-(len('subskip8')+1)]
    else:
        s2 = s[:-(len('subskip8')+3)]
    rename_dict[s] = s2
display(rename_dict)
#dfstat_short_nice.rename(lambda x: x.replace(rename_dict[x] ) , axis='columns')
dfstat_short_nice[prefix_role] = dfstat_short_nice[prefix_role].replace(rename_dict )
#dfstat_short_nice

# ########### rawnames rename
# rename_dict = {}
# for s in dfstat_short_nice[prefix_role]:
#     if prefix_role == 'prefix':
#         s2 = s[:-(len('subskip8')+1)]
#     else:
#         s2 = s[:-(len('subskip8')+3)]
#     rename_dict[s] = s2
# display(rename_dict)
#dfstat_short_nice.rename(lambda x: x.replace(rename_dict[x] ) , axis='columns')

assert( len( set(dfstat_short_nice['grouping'])  )  ) == 1
print(list(dfstat_short_nice['grouping'] ) [0] )
dfstat_short_nice = dfstat_short_nice.drop(['grouping'], axis=1)

rename_dict = {'onlyH':'all Hjorth parameters' , 'onlyH_act':'Hjorth activity', 
               'onlyH_mob':'Hjorth mobility', 'onlyH_compl':'Hjorth complexity', 'onlyH_modLFP':'all Hjorth parameters of LFP'}
dfstat_short_nice[prefix_role] = dfstat_short_nice[prefix_role].replace(rename_dict)

dfstat_short_nice['rawname'] = dfstat_short_nice['rawname'].apply(lambda x: x.upper())

# rename_dict = { 'merge_all_not_trem':'all non-tremor states', 
#             'merge_movements':'voluntary movements',
#             'merge_nothing':'none'          }
# dfstat_short_nice['grouping'].replace(rename_dict, inplace=True )

rename_dict = { 'basic':'tremor,quiet,hold,grasp', 
            'trem_vs_quiet':'tremor,quiet'  }
dfstat_short_nice['interval_set'] = dfstat_short_nice['interval_set'].replace(rename_dict )

# if not show_only_act:
#     dfstat_short_nice['num_mean'] = dfstat_short_nice['num_mean'].map(int)
    
pctf = lambda x: f'{x * 100:.0f}%' if isinstance(x,float) else x
pctize = ['bacc','tremor_det_perf','bacc_add',
          'tremor_det_perf_across']
for coln in pctize:
    dfstat_short_nice[coln] = dfstat_short_nice[coln].apply(pctf)
# dfstat_short_nice['performance / feature number'] = \
#     dfmn.apply(lambda row: f'{row.bacc_mean / row.num_mean:.4f}', axis=1 )

# rename column names
rename_dict = {}
rename_dict[prefix_role] = 'Feature set'
#rename_dict['grouping'] = 'behav. states merged'
rename_dict['interval_set'] = 'behav. states used'
rename_dict['bacc'] = 'performance'
rename_dict['bacc_add'] = 'performance across subj.'
rename_dict['tremor_det_perf'] = 'tremor detection'
rename_dict['tremor_det_perf_across'] = 'tremor detection across subj.'
# if not show_only_act:
#     rename_dict['num_mean'] = 'feature number'
dfstat_short_nice = dfstat_short_nice.rename(columns=rename_dict)

columns = np.array(list(dfstat_short_nice.columns) )[[0,1,2,5,6,3,4]]
columns

#dfstat_short_nice[columns]
#dfstat_short_nice = dfstat_short_nice.reset_index()
#dfstat_short_nice.style.set_properties(**{'text-align': 'left'})
dfstat_short_nice

#### Hjorth activity optimality (within other Hjorth)

In [ ]:
# optimality
dfopt = dfmn[(dfmn['grouping']=='merge_nothing') & (dfmn['interval_set']=='basic')].copy()

cols_desired = [prefix_role, 'tremor_det_perf_mean','tremor_det_perf_std',
               'bacc_mean','bacc_std','num_mean']
dfopt = dfopt[cols_desired]

dfopt['bacc_over_num'] = dfopt.apply(lambda row: \
                        f'{row.bacc_mean / row.num_mean:.3f}', axis=1)
dfopt.sort_values('bacc_over_num')

pctf = lambda x: f'{x * 100:.0f}%'
dfopt['num_mean'] = dfopt['num_mean'].map(int)
dfopt['bacc_mean'] = dfopt['bacc_mean'].apply(pctf )
dfopt['bacc_std'] = dfopt['bacc_std'].apply(pctf )
dfopt['tremor_det_perf_mean'] = dfopt['tremor_det_perf_mean'].apply(pctf )
dfopt['tremor_det_perf_std'] = dfopt['tremor_det_perf_std'].apply(pctf )

###############
for s in dfopt[prefix_role]:
    if prefix_role == 'prefix':
        s2 = s[:-(len('subskip8')+1)]
    else:
        s2 = s[:-(len('subskip8')+3)]
    rename_dict[s] = s2
display(rename_dict)
#dfstat_short_nice.rename(lambda x: x.replace(rename_dict[x] ) , axis='columns')
dfopt[prefix_role].replace(rename_dict, inplace=True )

rename_dict = {'onlyH':'all Hjorth parameters' , 'onlyH_act':'Hjorth activity', 
               'onlyH_mob':'Hjorth mobility', 'onlyH_compl':'Hjorth complexity'}
dfopt[prefix_role].replace(rename_dict, inplace=True )

rename_dict = {}
rename_dict[prefix_role] = 'Feature set'
rename_dict['grouping'] = 'behav. states merged'
rename_dict['interval_set'] = 'behav. states used'
rename_dict['bacc_mean'] = 'performance mean'
rename_dict['bacc_std'] = 'performance std'
rename_dict['tremor_det_perf_mean'] = 'tremor detection mean'
rename_dict['tremor_det_perf_std'] = 'tremor detection std'
rename_dict['num_mean'] = 'feature number'        
#
rename_dict['bacc_over_num'] = 'mean pef. / num. features'        
dfopt.rename(columns=rename_dict, inplace=True)



dfopt.style.set_properties(**{'text-align': 'left'})

#display( dfopt )

#### Compare groupings

In [ ]:
show_only_act = 1
if show_only_act:
    if prefix_role == 'prefix':
        good = 'onlyH_act_subskip8'
    else:
        good = 'onlyH_act_subskip8%%'
    dfstat_short_nice = dfstat_short[dfstat_short[prefix_role] == good].copy()
    dfstat_short_nice = dfstat_short_nice.drop(['num_mean'], axis=1)
else: # all Hjorth
    dfstat_short_nice = dfstat_short.copy()

rename_dict = {}
for s in dfstat_short_nice[prefix_role]:
    if prefix_role == 'prefix':
        s2 = s[:-(len('subskip8')+1)]
    else:
        s2 = s[:-(len('subskip8')+3)]
    rename_dict[s] = s2
display(rename_dict)
#dfstat_short_nice.rename(lambda x: x.replace(rename_dict[x] ) , axis='columns')
dfstat_short_nice[prefix_role].replace(rename_dict, inplace=True )


rename_dict = {'onlyH':'all Hjorth parameters' , 'onlyH_act':'Hjorth activity', 
               'onlyH_mob':'Hjorth mobility', 'onlyH_compl':'Hjorth complexity'}
dfstat_short_nice[prefix_role].replace(rename_dict, inplace=True )

rename_dict = { 'merge_all_not_trem':'all non-tremor states', 
            'merge_movements':'voluntary movements',
            'merge_nothing':'none'          }
dfstat_short_nice['grouping'].replace(rename_dict, inplace=True )

rename_dict = { 'basic':'tremor,quiet,hold,grasp', 
            'trem_vs_quiet':'tremor,quiet'  }
dfstat_short_nice['interval_set'].replace(rename_dict, inplace=True )

pctf = lambda x: f'{x * 100:.0f}%'
if not show_only_act:
    dfstat_short_nice['num_mean'] = dfstat_short_nice['num_mean'].map(int)
dfstat_short_nice['bacc_mean'] = dfstat_short_nice['bacc_mean'].apply(pctf )
dfstat_short_nice['bacc_std'] = dfstat_short_nice['bacc_std'].apply(pctf )
dfstat_short_nice['tremor_det_perf_mean'] = dfstat_short_nice['tremor_det_perf_mean'].apply(pctf )
dfstat_short_nice['tremor_det_perf_std'] = dfstat_short_nice['tremor_det_perf_std'].apply(pctf )

# dfstat_short_nice['performance / feature number'] = \
#     dfmn.apply(lambda row: f'{row.bacc_mean / row.num_mean:.4f}', axis=1 )

# rename column names
rename_dict = {}
rename_dict[prefix_role] = 'Feature set'
rename_dict['grouping'] = 'behav. states merged'
rename_dict['interval_set'] = 'behav. states used'
rename_dict['bacc_mean'] = 'performance mean'
rename_dict['bacc_std'] = 'performance std'
rename_dict['tremor_det_perf_mean'] = 'tremor detection mean'
rename_dict['tremor_det_perf_std'] = 'tremor detection std'
if not show_only_act:
    rename_dict['num_mean'] = 'feature number'
dfstat_short_nice.rename(columns=rename_dict, inplace=True)

columns = np.array(list(dfstat_short_nice.columns) )[[0,1,2,5,6,3,4]]
columns

dfstat_short_nice[columns]

In [ ]:
suff = '_subskip8'
d['onlyH'] =  'Hjorth'
for freq in ['tremor','beta','gamma']:
    d[f'allb_{freq}'] = f'{freq} interactions and band power w/o Hjorth'

In [ ]:
dfstat_short[dfstat_short['prefix'] == 'onlyH_act_subskip8']

In [ ]:
dfstat_short_nice.columns

In [ ]:
list(dfstat_short_nice.columns)

In [ ]:
We have compared 11 different feature sets: all Hjorth parameters together 

In [ ]:
dfstat_short

In [ ]:
% -- contra
^ -- ipsi



In [ ]:
#### Save perftable
ss = []
for tpl in to_show:
    ss += [','.join(tpl)]
s = ':'.join(ss)
print(s)

fn_table = pjoin(gv.data_dir,subdir, f'perftable_{s}_{len(prefixes)}prefs.npz')
print(fn_table)
svd = dict(table_info_per_perf_type=table_info_per_perf_type, table_per_perf_type=table_per_perf_type)
np.savez(fn_table, **svd)

In [ ]:
perf_to_use_list

### plot table info (many bars)

In [ ]:
#perf_tuple = perf_to_use_list[0]
#%debug
#for perf_tuple in perf_to_use_list[:1]:
for perf_tuple in perf_to_use_list:
    print(perf_tuple)
    postp.plotTableInfos2(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir,use_recalc_perf=False, 
                          prefixes_sorted=prefixes, crop_rawname=slice(None,None)) 
    plt.close()
    import gc;gc.collect()
    #break
#postp.plotTableInfos2(table_info_per_perf_type, perf_kind='LDA', keys = None, output_subdir=''): 

## load labels (not important where from exactly)
from utils import loadLabelsDict
labels_dict = loadLabelsDict()

In [ ]:
subdir

# Perf only

In [ ]:
perf_tuple = perf_to_use_list[-1]
defsp = 'special:min(sens,spec)'
score = 'bacc'

#TODO:  add part of plotCalcOutput
    #plt.title(plt.title())
scstr = ''
if score != defsp:
    scstr = score
fn_full = pjoin(gv.dir_fig,subdir,
                f'bars_perf_dif_subsets_{plotname_pref}_{scstr}.pdf')
plt.savefig(fn_full)

In [ ]:
prefixes

### H_act only

In [ ]:
from utils_postprocess_HPC import plotOnePrefQuick
pref_quick_plots = ['onlyH_act_subskip8BB']
for pref in pref_quick_plots:
    plotOnePrefQuick(rawnames, table_info_per_perf_type,perf_to_use_list[0], pref)

    plt.tight_layout()
    plt.savefig( pjoin(gv.dir_fig, subdir,f'{pref}_summary.pdf') )
    plt.close()

In [ ]:
from utils_postprocess_HPC import plotOnePrefQuick
pref_quick_plots = ['onlyH_act_modLFP_subskip8%%', 'onlyH_act_modLFP_subskip8^^', 
                      'onlyH_modLFP_subskip8%%', 'onlyH_modLFP_subskip8^^', 'onlyH_subskip8%%', 'onlyH_subskip8^^', 'onlyH_subskip8BB']
for pref in pref_quick_plots:
    plotOnePrefQuick(rawnames, table_info_per_perf_type,perf_to_use_list[0], pref)

    plt.tight_layout()
    plt.savefig( pjoin(gv.dir_fig, subdir,f'{pref}_summary.pdf') )
    plt.close()

In [ ]:
a = np.sort( np.unique( ords[1][ (ords[0]  == 'S07_on') & (ords[2]  == 'merge_nothing') ] ) )
#[aa for aa in a if aa.startswith()]
a

# Confmats

In [ ]:
'onlyH_subskip8' in prefixes

In [ ]:
# 'onlyH_subskip8',
#  'onlyH_subskip8BB',
#  'onlyH_subskip8BL',
#  'onlyH_subskip8BR',
#  'onlyH_subskip8LL',
#  'onlyH_subskip8RR']

In [ ]:
# Fig 1 -- 'onlyH_act_subskip8%%'

In [ ]:
#revdict_user = {'trem_L':0, 'notrem_L':1, 'hold_L':2, 'move_L':3}
#%debug
from utils_postprocess_HPC import filterOutputs
pref_confmat_plots = ['onlyH_modLFP_subskip8%%', 'onlyH_subskip8%%', 'onlyH_subskip8^^', 'onlyH_subskip8BB',
                      'onlyH_act_modLFP_subskip8%%', 'onlyH_act_modLFP_subskip8^^', 
                      'onlyH_modLFP_subskip8%%', 'onlyH_modLFP_subskip8^^', 
                      'onlyH_act_subskip8%%', 'onlyH_act_subskip8^^', 'onlyH_act_subskip8BB',
                     'LFPrel_noself_subskip8','LFPrel_noself_subskip8%%']
#                      'onlyH_subskip8%%', 'onlyH_subskip8^^', 'onlyH_subskip8BB']
#pref_confmat_plot = 'modLFP'
#pref_confmat_plot = 'onlyH_act_only15' # CB
#pref_confmat_plot = 'onlyH_act_only14' # CB
#pref_confmat_plot = 'onlyH_act_only0'  #Senosorimotor
#pref_confmat_plot = 'onlyH_act_LFPand_only14' # CB
#pref_confmat_plot = 'onlyH_act_LFPand_only0'  #Senosorimotor
import pandas as pd
from globvars import gp

colnames_add = []
for it_mvt in ['hold', 'move','trem']:
    colnames_add += [it_mvt + '_perf_mean']
    colnames_add += [it_mvt + '_perf_std']

dfcm = pd.DataFrame(columns = ['prefix_templ', 'grouping','interval_set','confmats','confmats_offdiag',
                              'confmats_diag'] + colnames_add)
for pref_confmat_plot in pref_confmat_plots:
    for grp,it in [('merge_nothing','basic')]: #, 'merge_movements']:
        print(pref_confmat_plot)
        #grps = ['merge_nothing'] #, 'merge_movements']
        #grps = [ 'merge_movements']
        outputs_filtered = postp.filterOutputs(output_per_raw,prefs=[pref_confmat_plot], 
                                               grps=[grp], its=[it])
        plt.rcParams.update({'font.size': 15})
        plt.rc('ytick',labelsize=22)
        plt.rc('xtick',labelsize=22)
        plt.rc('axes',labelsize=24)

        colorbar_axes_bbox = [0.80, 0.2, 0.025, 0.7]
        _,_,confmats_normalized,confmat_normalized_offdiags, confmat_normalized_diags_els = \
            postp.plotConfmats(outputs_filtered, ww = 5, hh =5, keep_beh_state_sides=0,
                           keep_subj_list_title=1,
                           labelpad_cbar=155, colorbar_axes_bbox= colorbar_axes_bbox,
                          rename_class_names = {'notrem':'quiet'})
        
        d = {'confmats':confmats_normalized, 'confmats_offdiag':confmat_normalized_offdiags,
            'confmats_diag': confmat_normalized_diags_els}
        d['grouping'] = grp
        d['interval_set'] = it
        d['prefix_templ'] = pref_confmat_plot
        #plt.gcf().axes[-1].set_visible(False)
        figname = f'confmats_{pref_confmat_plot}_{grp},{it}.pdf'
        figname_full = pjoin(gv.dir_fig,subdir,figname)
        plt.savefig(figname_full)
        print(figname_full)
        plt.close()
        
        confmats_all = np.array( confmats_normalized )
        #confmats_all.shape
        cmm = confmats_all.mean(axis=0)
        cmstd = confmats_all.std(axis=0)        
        # diag el
        for it_mvt in ['hold', 'move','trem']:
            i = gp.int_types_basic.index(it_mvt)    
            me = cmm[i,i]
            std = cmstd[i,i]
            d[it_mvt + '_perf_mean'] = me
            d[it_mvt + '_perf_std'] = std
        
        # this is averaged over subjects/medcond 
        dfcm = dfcm.append(d,ignore_index = 1)
dfcm.reset_index()

In [ ]:
dfcm[['prefix_templ','grouping','interval_set','trem_perf_mean','trem_perf_std']]

In [ ]:
row = df.loc[0]
row['interval_set']
row['grouping']
row['perf_dict_shuffled']['confmat']
#getConfmat()

### Prpare funtions

In [ ]:
def prefixTempl2Prefix(prefix_templ,rawname):
    subj = rawname.split('_')[0]
    mainmoveside = gv.gen_subj_info[subj].get('move_side',None)
    assert mainmoveside is not None
    movesidelet = mainmoveside[0].upper()
    moveopsidelet = utils.getOppositeSideStr( movesidelet )
    templ_eff = prefix_templ.    replace('%', moveopsidelet)
    templ_eff = templ_eff.replace('^', movesidelet)
    return templ_eff


def getMcf(row):
    if 'rawname' not in row:
        rawnames_cur = rawnames
    else:
        rawname = row['rawname']
        rawnames_cur = [rawname]
    res = []
    for rawname in rawnames_cur:
        if 'prefix' in row:
            prefix = row['prefix']
        else:
            prefix = prefixTempl2Prefix(row['prefix_templ'],rawname)
        r = output_per_raw[rawname][prefix][row['grouping']][row['interval_set']]
        res += [r]
    return res
        
def getShuffledConfmatsStats(row):
    r = getMcf(row)

    cms = []
    for mcf in r:
        cm,lblord = getConfmat(mcf,row['grouping'],row['interval_set'],shuffled=1)
        cms += [cm]
    cms = np.array(cms)
    cms_mn = cms.mean(axis=0)
    cms_std = cms.std(axis=0)
    return cms_mn,cms_std
    
row = dfcm_sub.reset_index().loc[0]
cms_mn,cms_std = getShuffledConfmatsStats(row)


def getCM(row, data_type, stat):
    if data_type == 'normal':
        confmats_normalized =  row['confmats']
        confmats_all = np.array(confmats_normalized)
        cms_mn = confmats_all.mean(axis=0)
        cms_std = confmats_all.std(axis=0)
    else:
        cms_mn,cms_std = getShuffledConfmatsStats(row)
            
    if stat == 'mean':
        cms = cms_mn
    elif stat == 'std':
        cms = cms_std
    return cms

def getStat(row, data_type, stat, it_mvt):
    cms = getCM(row, data_type, stat)    
    i = gp.int_types_basic.index(it_mvt) 
    return cms[i,i]
    
def getBest(row, data_type, best_type, stat=None):
    cms = getCM(row, data_type, 'mean')
    assert cms.ndim == 2
    
    eye = np.eye(cms.shape[0])
    diag = np.diag(np.diag(cms))
    cms_offdiag = cms - diag
    cms_offdiag_largedval = cms_offdiag + eye * 1e5
    
    if best_type == 'best':
        ip = np.unravel_index( np.argmin( np.mean( cms_offdiag_largedval, 0)  ), cmm.shape )    
    elif best_type == 'worst':
        ip = np.unravel_index( np.argmax( np.mean( confmat_normalized_offdiags, 0)  ), cmm.shape )
    
    if stat is None:
        s = f'{gp.int_types_basic[ip[0]]} pred as {gp.int_types_basic[ip[1]]}'
        return s
    if stat == 'mean':
        return cms[tuple(ip)]
    elif stat == 'std':
        cms_std = getCM(row, data_type, 'std')
        return cms_std[tuple(ip)]
    # f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)' )

### fill confusion matrix data for pub

In [ ]:
cms = []
for mcf in r:
    cm,lblord = getConfmat(mcf,row['grouping'],row['interval_set'],shuffled=1)
    cms += [cm]
cms = np.array(cms)
cms_mn = cms.mean(axis=0)
cms_std = cms.std(axis=0)
cms_mn,cms_std

In [ ]:
def getCM(row, data_type, stat):
    if data_type == 'normal':
        confmats_normalized =  row['confmats']
        confmats_all = np.array(confmats_normalized)
        cms_mn = confmats_all.mean(axis=0)
        cms_std = confmats_all.std(axis=0)
    else:
        cms_mn,cms_std = getShuffledConfmatsStats(row)
            
    if stat == 'mean':
        cms = cms_mn
    elif stat == 'std':
        cms = cms_std
    return cms

def getStat(row, data_type, stat, it_mvt):
    cms = getCM(row, data_type, stat)    
    i = gp.int_types_basic.index(it_mvt) 
    return cms[i,i]
    
def getBest(row, data_type, best_type, stat=None):
    cms = getCM(row, data_type, 'mean')
    assert cms.ndim == 2
    
    eye = np.eye(cms.shape[0])
    diag = np.diag(np.diag(cms))
    cms_offdiag = cms - diag
    cms_offdiag_largedval = cms_offdiag + eye * 1e5
    
    if best_type == 'best':
        ip = np.unravel_index( np.argmin( np.mean( cms_offdiag_largedval, 0)  ), cmm.shape )    
    elif best_type == 'worst':
        ip = np.unravel_index( np.argmax( np.mean( confmat_normalized_offdiags, 0)  ), cmm.shape )
    
    if stat is None:
        s = f'{gp.int_types_basic[ip[0]]} pred as {gp.int_types_basic[ip[1]]}'
        return s
    if stat == 'mean':
        return cms[tuple(ip)]
    elif stat == 'std':
        cms_std = getCM(row, data_type, 'std')
        return cms_std[tuple(ip)]
    # f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)' )

In [ ]:
for data_type in ['normal','shuffled']:
    for stat in ['mean','std']:
        for it_mvt in ['hold', 'move','trem']:
            fcur = lambda row : getStat(row, data_type, stat, it_mvt)
            res = dfcm.apply(fcur, axis=1)
            dfcm[f'detperf_{data_type}_{it_mvt}_{stat}'] = res                        
            
    if data_type != 'shuffled':            
        for best_type in ['best', 'worst']:            
            fcur = lambda row : getBest(row, data_type, best_type=best_type, stat=None)
            res = dfcm.apply(fcur, axis=1)
            dfcm[f'{best_type}_pair'] = res                        

            for stat in ['mean','std']:
                fcur = lambda row : getBest(row, data_type, best_type=best_type, stat=stat)
                res = dfcm.apply(fcur, axis=1)
                dfcm[f'{best_type}_pair_{stat}'] = res                                        
            
dfcm

In [ ]:
cols = ['prefix_templ',
 'detperf_normal_hold_mean',
 'detperf_normal_move_mean',
 'detperf_normal_trem_mean',
 'detperf_normal_hold_std',
 'detperf_normal_move_std',
 'detperf_normal_trem_std',
 'detperf_shuffled_hold_mean',
 'detperf_shuffled_move_mean',
 'detperf_shuffled_trem_mean',
 'detperf_shuffled_hold_std',
 'detperf_shuffled_move_std',
 'detperf_shuffled_trem_std',
 'best_pair',
 'best_pair_mean',
 'best_pair_std',
 'worst_pair',
 'worst_pair_mean',
 'worst_pair_std']

cols1 = ['prefix_templ',
 'detperf_normal_hold_mean',
 'detperf_normal_move_mean',
 'detperf_normal_trem_mean',
 'detperf_normal_hold_std',
 'detperf_normal_move_std',
 'detperf_normal_trem_std',
 'detperf_shuffled_hold_mean',
 'detperf_shuffled_move_mean',
 'detperf_shuffled_trem_mean',
 'detperf_shuffled_hold_std',
 'detperf_shuffled_move_std',
 'detperf_shuffled_trem_std']

cols2 =  ['prefix_templ','best_pair',
 'best_pair_mean',
 'best_pair_std',
 'worst_pair',
 'worst_pair_mean',
 'worst_pair_std']

cols_nice = []
for col in cols:
    if col.find('detperf_normal') >= 0:
        col_nice = col.replace('detperf_normal_','')
    else:
        col_nice = col
    cols_nice += [col_nice]
rename_dict = dict( zip(cols,cols_nice) )

In [ ]:
list( dfcm_sub['hold_mean'] )


In [ ]:
interval_names_pub = {'notrem':'quet','move':'grasp','trem':'tremor','hold':'hold'}

In [ ]:
# show dataframes for best/worst stats AND print human readable
for pref_templ_desired in ['onlyH_act_subskip8%%', 'onlyH_act_modLFP_subskip8%%']:
    dfcm_sub = dfcm[dfcm['prefix_templ']==pref_templ_desired  ]
    assert len(dfcm_sub) == 1
    #dfcm_sub = dfcm_sub[cols]
    dfcm_sub = dfcm_sub[cols2]
    #dfcm_sub = dfcm_sub[cols1]
    dfcm_sub = dfcm_sub.rename(rename_dict, axis='columns')
    display(dfcm_sub)
    
    s = ''
    
    bp  = list( dfcm_sub[f'best_pair'] )[0]
    me  = list( dfcm_sub[f'best_pair_mean'] )[0]
    std = list( dfcm_sub[f'best_pair_std'] )[0]
    
    for it in interval_names_pub:
        if bp.find(it) >= 0:
            bp = bp.replace(it,f'\'{interval_names_pub[it]}\'' )
    bp = bp.replace('pred as','predicted to be')
    s += f'Least often detected confusion: {bp} ('
    s += f'mean={me:.0f}%, std={std:.0f}%). '
    
    bp  = list( dfcm_sub[f'worst_pair'] )[0]
    me  = list( dfcm_sub[f'worst_pair_mean'] )[0]
    std = list( dfcm_sub[f'worst_pair_std'] )[0]
    
    for it in interval_names_pub:
        if bp.find(it) >= 0:
            bp = bp.replace(it,f'\'{interval_names_pub[it]}\'' )
    
    bp = bp.replace('pred as','predicted to be')
    s += f'Most often detected confusion: {bp} ('
    s += f'mean={me:.0f}%, std={std:.0f}%); '
    
    print(s)

In [ ]:
# show dataframes for main stats
for pref_templ_desired in ['onlyH_act_subskip8%%', 'onlyH_act_modLFP_subskip8%%']:
    dfcm_sub = dfcm[dfcm['prefix_templ']==pref_templ_desired  ]
    assert len(dfcm_sub) == 1
    #dfcm_sub = dfcm_sub[cols]
    #dfcm_sub = dfcm_sub[cols2]
    dfcm_sub = dfcm_sub[cols1]
    dfcm_sub = dfcm_sub.rename(rename_dict, axis='columns')
    display(dfcm_sub)
    


In [ ]:
# print human readable
for pref_templ_desired in ['onlyH_act_subskip8%%', 'onlyH_act_modLFP_subskip8%%']:
    dfcm_sub = dfcm[dfcm['prefix_templ']==pref_templ_desired  ]
    assert len(dfcm_sub) == 1
    #dfcm_sub = dfcm_sub[cols]
    #dfcm_sub = dfcm_sub[cols2]
    dfcm_sub = dfcm_sub[cols1]
    #dfcm_sub = dfcm_sub.rename(rename_dict, axis='columns')
    #display(dfcm_sub)
    
    row = dfcm_sub.reset_index().loc(0)    
    #for data_type in ['normal', 'shuffled']:
    print(f'  {data_type}')
    for it_mvt in ['trem', 'hold', 'move']:
        s = ''
        #i = gp.int_types_basic.index(it_mvt)    
        #me = cmm[i,i]
        #std = cmstd[i,i]
        data_type = 'normal'
        me  = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_mean'] )[0]
        std = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_std'] )[0]
        
        data_type = 'shuffled'
        me_s  = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_mean'] )[0]
        std_s = list( dfcm_sub[f'detperf_{data_type}_{it_mvt}_std'] )[0]
        s += (f'for \'{interval_names_pub[it_mvt]}\' movement the '
            'statistic of correct detection is ')
        s += f'mean={me:.0f}, std={std:.0f}; '
        s += ' (for shuffled data '
        s += f'mean={me_s:.0f}, std={std_s:.0f}); '
        print(s)

In [ ]:
dfcm_sub.rename(rename_dict, axis='columns')

In [ ]:
dfcm[dfcm['prefix_templ']==pref_templ_desired  ]

In [ ]:
# to put in Discussion
For all Hjorth parameters together

### Worst and best

In [ ]:
len(dfcm[dfcm['prefix_templ']==pref_templ_desired  ])

In [ ]:
for pref_templ_desired in ['onlyH_act_subskip8%%', 'onlyH_act_modLFP_subskip8%%']:
    print('   ' ,pref_templ_desired)
    dfcm_sub = dfcm[dfcm['prefix_templ']==pref_templ_desired  ]
    assert len(dfcm_sub) == 1
    confmats_normalized =  dfcm_sub['confmats']
    confmats_normalized = list(confmats_normalized)[0]
    assert len(confmats_normalized) == 11
    
    row = dfcm_sub.reset_index().loc[0]
    cms_mn,cms_std = getShuffledConfmatsStats(row)

    from globvars import gp
    confmats_all = np.array( confmats_normalized )
    #confmats_all.shape
    cmm = confmats_all.mean(axis=0)
    cmstd = confmats_all.std(axis=0)

    eyes = np.array( [np.eye(confmats_all.shape[-1] ) for cm in confmats_normalized], dtype=bool)
    confmat_normalized_offdiags_largedval = confmat_normalized_offdiags + eyes * 1e5

    ip = np.unravel_index( np.argmax( np.mean( confmat_normalized_offdiags, 0)  ), cmm.shape )
    print(f'Most often wrong attribution -- {gp.int_types_basic[ip[0]]} is predicted to be {gp.int_types_basic[ip[1]]}, '
    f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)')

    ip = np.unravel_index( np.argmin( np.mean( confmat_normalized_offdiags_largedval, 0)  ), cmm.shape )
    print(f'Least often wrong attibution -- {gp.int_types_basic[ip[0]]} is predicted to be {gp.int_types_basic[ip[1]]}, '
     f'(mean={cmm[tuple(ip)]:.0f}%, std={cmstd[tuple(ip)]:.0f}%)' )

    s = ''
    for it_mvt in ['hold', 'move','trem']:
        i = gp.int_types_basic.index(it_mvt)    
        me = cmm[i,i]
        std = cmstd[i,i]
        s += f'for \'{it_mvt}\' movement the statistic of correct detection is '
        s += f'mean={me:.0f}, std={std:.0f}; \n'
        
    
    print(s)

In [ ]:
#prefixes

# onlyBar

In [ ]:
table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(output_per_raw, prefixes=prefixes, 
    perf_to_use_list=perf_to_use_list, to_show=to_show)

## prepare plotspecs

In [ ]:
plotspecs = {}  # plot name 2 stuff
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP contralat',
                     'onlyH_act_modLFP_subskip8^^':'LFP ipsilat',
                     'onlyH_act_modLFP_subskip8BB':'LFP bilat',
                     'onlyH_act_only_best_among_single-sided': 'best area SS',
                     'onlyH_act_LFPand_quasibest_among_single-sided': '*LFP + best area SS',
                     'onlyH_act_LFPand_best_among_single-sided': 'LFP + best area SS',
                     'onlyH_act_only_best_among_clmove': 'best area CLM',
                     'onlyH_act_LFPand_quasibest_among_clmove': '*LFP + best area CLM',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area CLM',
                     'onlyH_act_only_best_among_two-sided': 'best area TS',
                     'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area TS',
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP + best area TS',
                     good_prefs_permod['msrc'][0]:'cortex',
                    'onlyH_act_subskip8%%':'LFP+cortex contralat',
                     'onlyH_act_subskip8^^':'LFP+cortex ipsilat',
                     'onlyH_act_subskip8BB':'LFP+cortex bilat'}
plotspecs['all'] = prefix2final_name, 4,8  # numbers are width and height
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP contralat',
                     'onlyH_act_modLFP_subskip8^^':'LFP ipsilat',
                     'onlyH_act_modLFP_subskip8BB':'LFP bilat'}
plotspecs['onlyBestLFP'] = prefix2final_name, 1.8,5
prefix2final_name = { 'onlyH_act_only_best_among_two-sided': 'best area TS',
                     'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area TS',
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP + best area TS' }
plotspecs['two-sided'] = prefix2final_name, 1.8,5
prefix2final_name = { 'onlyH_act_only_best_among_single-sided': 'best area SS',
                     'onlyH_act_LFPand_quasibest_among_single-sided': '*LFP + best area SS',
                     'onlyH_act_LFPand_best_among_single-sided': 'LFP + best area SS' }
plotspecs['single-sided'] = prefix2final_name, 1.8,5
prefix2final_name = {'onlyH_act_only_best_among_clmove': 'best area CLM',
                     'onlyH_act_LFPand_quasibest_among_clmove': '*LFP + best area CLM',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area CLM' }
plotspecs['contralat'] = prefix2final_name, 1.8,5
prefix2final_name = {'onlyH_act_only_best_among_clmove': 'best area CLM',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP + best area CLM',
                     'onlyH_act_only_best_among_ilmove': 'best area ILM',
                     'onlyH_act_LFPand_best_among_ilmove': 'LFP + best area ILM' }
plotspecs['contr_vs_ipsi'] = prefix2final_name, 1.8,5

In [ ]:
np.array( list(confmats_normalized)[0] ).shape

In [ ]:
plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',
                     'onlyH_act_only_best_among_clmove': 'best area',
                     'onlyH_act_LFPand_best_among_clmove': 'LFP+best area',
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',                     
                     'onlyH_act_subskip8%%':'LFP+cortex',
                     'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
plotspecs['contra_various'] = prefix2final_name, 2.8,6

In [ ]:
# Fig 3
plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',  
                     'onlyH_act_only_best_among_two-sided': 'best area (two-sided)',                                         
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
                     'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
#'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
#'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area (two-sided)',
plotspecs['contra_various_TSonly_wandwoLFP'] = prefix2final_name, 2.2,6

In [ ]:
len(confmats_normalized)

In [ ]:
# Fig 3 (not in manuscript) with worse as well
plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',  
                     'onlyH_act_only_best_among_two-sided': 'best area (two-sided)',                                         
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
                     'onlyH_act_only_worst_among_two-sided': 'worst area (two-sided)',                                         
                     'onlyH_act_LFPand_worst_among_two-sided': 'LFP+worst area (two-sided)',
                     'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
#'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
#'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area (two-sided)',
plotspecs['contra_various_TSonly_wandwoLFP_inc_worst'] = prefix2final_name, 2.2,7

In [ ]:
#plotspecs = {}  
prefix2final_name = {'onlyH_act_modLFP_subskip8%%':'LFP',  
                     'onlyH_act_only_best_among_two-sided': 'best area (two-sided)',                                          
                     'onlyH_act_subskip8BB':'LFP+cortex (two-sided)'}
#'onlyH_act_LFPand_best_among_two-sided': 'LFP+best area (two-sided)',
plotspecs['contra_various_TSonly'] = prefix2final_name, 2.2,6

In [ ]:
plotspecs = {}  # plot name 2 stuff
prefix2final_name = { 'onlyH_act_subskip8%%':'Hjorth act contralat',
                     'onlyH_act_subskip8^^':'Hjorth act ipsilat',
                     'onlyH_act_subskip8BB':'Hjorth act bilat',
                     'onlyH_subskip8%%':'Hjorth act,mob,compl contralat',
                     'onlyH_subskip8^^':'Hjorth act,mob,compl ipsilat',
                     'onlyH_subskip8BB':'Hjorth act,mob,compl bilat'}
plotspecs['act_vs_all'] = prefix2final_name, 2.1,6


In [ ]:
plotspecs = {}  # plot name 2 stuff
prefix2final_name = { 'onlyH_act_only_best_among_two-sided': 'best area TS',
                     'onlyH_act_LFPand_quasibest_among_two-sided': '*LFP + best area TS',
                     'onlyH_act_LFPand_best_among_two-sided': 'LFP + best area TS' }
plotspecs['two-sided'] = prefix2final_name, 1.8,5


In [ ]:
prefix2final_name

In [ ]:
perf_tuple[:-1]

In [ ]:
plotspecs, len(table_info_per_perf_type)

## run plot gen

In [ ]:
from utils_postprocess_HPC import addBestParcelGroups
#plotname_pref = 'noLFP'
plotname_pref = ''
add_pg  = False

#%debug

for plotspec_name,(prefix2final_name,hh,ww) in plotspecs.items():
    score = 'bacc'
    prefixes_final = list( prefix2final_name.keys() )
    for perf_tuple in table_info_per_perf_type:
        print(f'Starting plotTableInfos_onlyBar for {perf_tuple}')
        if add_pg:
            addBestParcelGroups(output_per_raw, table_info_per_perf_type, perf_tuple, score , do_add=1, 
                            remove_redundant_quasibest = False)

        plt.rc('axes', titlesize=18)
        plt.rc('axes', labelsize=20)
        plt.rc('ytick', labelsize=16)
        plt.rc('xtick', labelsize=16)
        
        k = list( table_info_per_perf_type.keys() )[0]
        axs, df = postp.plotTableInfos_onlyBar(table_info_per_perf_type,
                                           perf_tuple=perf_tuple,
                              output_subdir=subdir,use_recalc_perf=False,
                              prefixes_sorted=prefixes_final, prefix2final_name=prefix2final_name,
                                     crop_rawname='no',
                                           score= score,
                                           rawnames=rawnames, per_medcond =1,
                                          expand_best = 1, red='shuffled',
                                           allow_missing_prefixes = 1, hh =hh ,ww =ww)
        #axs[0,0].set_xlabel('')
        axs[2,1].set_visible(False)
        #frame1.axes.get_yaxis().set_visible(False)


    defsp = 'special:min(sens,spec)'
    if score != defsp:
        scstr = score
    fn_full = pjoin(gv.dir_fig,subdir, f'bars_perf_{plotspec_name}_{plotname_pref}_{scstr}.pdf')
    plt.subplots_adjust(hspace=0.2)

    plt.tight_layout()
    plt.savefig(fn_full)
    plt.close()


## Print collected stats

In [ ]:
list ( df['barname_pre_human'] )

In [ ]:
barnames = set( df['barname_pre_human'] ) # indep of subject names
print(barnames)
r = {}
rstd = {}
s = f'Performance statistics are: '
s_shuffled = f'Performance statistics for shuffled data are: '
for bn in barnames:
    dfsub = df[ df['barname_pre_human'] == bn ]
    me  = dfsub['p'].mean()
    std = dfsub['p'].std()
    r[bn] = me
    rstd[bn] = std
    s += f'{bn}: mean={me:.2f}%, std={std:.2f}%; '
    
    me  = dfsub['p_red'].mean()
    std = dfsub['p_red'].std()
    s_shuffled += f'{bn}: mean={me:.2f}%, std={std:.2f}%; '
r
s = s[:-2] + '.'
print(s)
print(s_shuffled)
display(r)

In [ ]:
s = 'shows that best STN LFP channel alone gives poor performance: '
bn = 'LFP'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 
##
s += 'Best сortical areas show higher performance than the best LFP channel: '
bn = 'best area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 

s += 'Adding LFP features improves performance a bit further: '
bn = 'LFP+best area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 

s += 'Using an entire entire cortex gives even better performance :'
bn = 'LFP+cortex (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. ' 

s += ('All feature subsets for every subject show performance '
'better than performance of the classifier trained on shuffled labels. ')
s += s_shuffled

s = s[:-2] + '.'
s

In [ ]:
s = 'Worst area performance had '
bn = 'worst area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. '

s += 'Adding LFP to it gave us performance with '
bn = 'LFP+worst area (two-sided)'; me = r[bn]; std = rstd[bn]
s += f'mean={me:.2f}%, std={std:.2f}%. '

s = s[:-2] + '.'
print(s)

# Only one area

In [ ]:
prefixes

In [ ]:
corresp

In [ ]:
#tpl[-1]['cmd']
 #[]

## Filtering to leave one-sided and not-pgn-like

In [ ]:
tpll_new = []
basepref1 = 'onlyH_act_only'
basepref2 = 'onlyH_act_LFPand_only'
for tpl in tpll:
    prefix = tpl[1]
    g = 0
    d = dict( tpl[-1]['cmd'][0] )
    pgn = d.get('--parcel_group_names' ,None)
    if prefix.startswith(basepref1) or prefix.startswith(basepref2):        
        assert pgn is not None
        if pgn.endswith('_L') or pgn.endswith('_R'):
            tpll_new += [tpl]
            g = 1
    else:
        tpll_new += [tpl]
        g = 2
    #tpl[1]['cmd']
#     corresp,all_info = loadRunCorresp(tpl[-1])
#     ind,pgn2,nice_name = corresp.get(prefix, (None,None,None) )
#     if pgn2 is not None:
#         if pgn.endswith('_L') and pgn.endswith('_R'):
#             tpll_new += [tpl]
#         else:            
#             if pgn2 is not None:
#                 continue
    if g:
        print(g,prefix,pgn
    
#outputs_filtered0 = tupleList2multiLevelDict(tpll_new)

In [ ]:
#tpll[0][:-1]

In [ ]:
mco = tpll[0][-1]
from utils_postprocess_HPC import computeImprovementsPerParcelGroup2,plotTableInfoBrain
from utils_postprocess_HPC import filterOutputs, tupleList2multiLevelDict
#%debug
prefs =  [pref for pref in prefixes if pref.startswith('onlyH_act_only') ]
prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_subskip8BB') ]
prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_modLFP_subskip8') ]
#prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_exclude') ]
#prefs += ['onlyH_act']
base_perf_prefix = 'onlyH_act_subskip8BB'
#base_perf_low_prefix = 'onlyH_act_modLFP_subskip8%%'
base_perf_low_prefix = None

#outputs_filtered0 = tupleList2multiLevelDict(tpll_new)
outputs_filtered = postp.filterOutputs(outputs_filtered0, rns=rawnames , prefs=prefs, grps = ['merge_nothing'] )
checkTupleListTableCompleteness(outputs_filtered)
#make_plots = False
make_plots = False
prefs_after_filter = {}
impr_per_rn = {}
infos = []
for rn in rawnames:
#for rn in ['S01_on', 'S02_on']:
    outputs_filtered = postp.filterOutputs(outputs_filtered0, rns=[rn] , prefs=prefs, grps = ['merge_nothing'] )
    checkTupleListTableCompleteness(outputs_filtered)
    medcond = rn.split('_')[1]
    prefs_after_filter[rn] = list(outputs_filtered[rn].keys())
#    for mode in ['only','exclude']:
    for mode in ['only']:
        print(f'-------  Starting {rn} {medcond}  {mode}')
        impr_wrt_base_per_medcond_per_pgn=computeImprovementsPerParcelGroup2(outputs_filtered, 
                 base_perf_prefix = base_perf_prefix, base_perf_low_prefix = base_perf_low_prefix,
                 mode = mode, inv_exclude = True,
                 score = 'balanced_accuracy',
                 ignore_base_prefix_missing=1)
#         impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn,\
#         impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
#             computeImprovementsPerParcelGroup(outputs_filtered, 
#                  'onlyH_act', mode = mode, inv_exclude = True,
#                  score = 'balanced_accuracy',
#                  ignore_base_prefix_missing=1)
#         impr_per_rn[rn] = impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn,\
#             impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond
        impr_per_rn[rn] = impr_wrt_base_per_medcond_per_pgn
    #        return impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond
        #break
        for medcond in impr_wrt_base_per_medcond_per_pgn.keys():
            if len(impr_wrt_base_per_medcond_per_pgn[medcond] ) <= 4:
                print(f'WARNING: {rn} {medcond} has zero len impr_wrt_base_per_medcond_per_pgn[{medcond}]')
                continue            
            axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                   head_subj_ind=None, mode=mode, subdir=subdir,
                    savefile_prefix=f'EXPORT_brain_map_area_strength_{rn}_',
                                               save_only = not make_plots)
            infos += [(mode,medcond,info)]
            if make_plots:
                plt.close()
            #break
        #break

In [ ]:
#impr_wrt_base_per_medcond_per_pgn

In [ ]:
set(prefs_after_filter['S01_on'] ) ^ set( prefs_after_filter['S02_on'] )

rn1 = 'S01_on'
outputs_filtered1 = postp.filterOutputs(output_per_raw, rns=[rn1] , prefs=prefs, grps = ['merge_nothing'] )
l1 = list(outputs_filtered1[rn1].keys())
rn2 = 'S02_on'
outputs_filtered2 = postp.filterOutputs(output_per_raw, rns=[rn2] , prefs=prefs, grps = ['merge_nothing'] )
l2 = list(outputs_filtered2[rn2].keys())
set(l1) ^ set(l2)

'2 ' *2

list(sorted( output_per_raw['S02_on'].keys() ) )

outputs_filtered2['S02_on']['onlyH_act_only15'].keys()

oo=0
tpll2 = pp.multiLevelDict2TupleList(outputs_filtered2,4,3)
tpll_reshaped2 = list( zip(*tpll2) )
oo=0
tpll1 = pp.multiLevelDict2TupleList(outputs_filtered1,4,3)
tpll_reshaped1 = list( zip(*tpll1) )

len( outputs_filtered2[rn2] ), len( outputs_filtered1[rn1] )

len( tpll_reshaped2[1] ), len( tpll_reshaped1[1] )

list ( zip(l1,l2) )

prefs_after_filter['S02_on']

list( outputs_filtered['S02_on'].keys() )

m = (tpll_reshaped[0] == 'S02_on') & (tpll_reshaped[2] == 'merge_nothing')
list ( tpll_reshaped[1][m] ) , list( tpll_reshaped[3][m] )

rn = 'S07_on'
aa = impr_per_rn[rn][rn.split('_')[-1]]

impr_per_rn

In [ ]:
order = ['LFP', 'Sensorimotor', 'Cerebellum',
    'FrontalMed',
 'TemporalMid',
 'SupraMarginal',
 'OccipitalInf',
 'Angular',
 'FrontalInf',
 'ParietalSup',
 'TemporalSup',
 'FrontalSup',
 'TemporalInf',
 'OccipitalSup',
 'OccipitalMid',
 'ParietalInf']
keyorder = lambda tpl: order.index( tpl[0] )

## Make hor barplots one per subj, one bar per area, each medcond dif color

In [ ]:
subjs = list(sorted(set( [rn.split('_')[0] for rn in rawnames] ) ))
#dict_keys(['sens_recalc', 'spec_recalc', 'descr', 'comment_from_runstrings', 'sens', 'spec', 
# 'F1', 'acc', 'bacc', 'sens_red', 'spec_red', 'sens_red_recalc', 'spec_red_recalc', 'F1_red', 'num', 'num_red'])
aa = table_info_per_perf_type[perf_to_use_list[0]]
tpls = sorted( list(aa.keys() ), key=lambda x: x[0] )

nr = len(subjs)
nc = 1
ww = 5; hh = 6
fig,axs = plt.subplots(nr,nc,figsize=( nc*ww, nr*hh), sharex='col')
plt.subplots_adjust(top=0.97,bottom=0.02,left=0.25,right=0.99,hspace=0.1)

color_per_medcond = dict(ON='blue',OFF='red')

for rn,imprs in impr_per_rn.items():
    #rn,grp,it = tpl
    subj,medcond = rn.split('_')
    axi = subjs.index(subj)
    impr_cur = imprs[medcond]
    
    medcond = medcond.upper()
    srt = sorted(impr_cur.items(), key=keyorder)
    srt = list(srt)[::-1]
    ks,vs=  zip(*srt)
    
    ax = axs[axi]
    ax.barh(ks,vs, color=color_per_medcond[medcond], alpha=0.7 )
    
    print(rn,axi)
    print(rn,srt)
    
    #rpint(aa[tpl].keys() )
    #print( aa[tpl]['onlyH_act'].keys() )
    
    #val = aa[tpl]['onlyH_act']['bacc']
    #print(rn, axi, val )
    
    #ax.barh([medcond ],[val] )
    ax.set_xlim(0,100)
    ax.set_title(subj)
    ax.set_xlabel('balanced acc')
    #break
#.keys()
#plt.tight_layout()
plt.suptitle('Perf per brain area (red=OFF, blue=ON)', y=0.99)
plt.savefig( pjoin(gv.dir_fig, subdir,'H_act_perf_summary_per_area.pdf') )
plt.close()

In [ ]:
#impr_wrt_base_per_medcond_per_pgn

In [ ]:
axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                               head_subj_ind=None, mode=mode, subdir=subdir,
                                               savefile_prefix=f'EXPORT_brain_map_area_strength_{rn}_')

In [ ]:
impr_wrt_base_per_medcond_per_pgn

# Improvement compared to LFP

In [ ]:
mco = tpll[0][-1]
from utils_postprocess_HPC import computeImprovementsPerParcelGroup2,computeImprovementsPerParcelGroup,plotTableInfoBrain

base_perf_prefix = 'onlyH_act_subskip8BB'
base_perf_low_prefix = 'onlyH_act_modLFP_subskip8%%'


pref_LFPand = [pref for pref in prefixes if pref.startswith('onlyH_act_LFPand_') ]
pref_LFPand += [base_perf_prefix, base_perf_low_prefix]

#outputs_filtered0 = tupleList2multiLevelDict(tpll_new)
make_plots = False

#%debug
ress = []
for rn in rawnames:
    outputs_filtered = postp.filterOutputs(outputs_filtered0, rns=[rn], prefs=pref_LFPand, grps = ['merge_nothing'])
    medcond = rn.split('_')[1]
    #for mode in ['LFPand_only','LFPand_exclude']:
    for mode in ['LFPand_only']:
        print(f'-------  Starting {rn} {medcond}  {mode}')
#         impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
#             computeImprovementsPerParcelGroup(outputs_filtered, 
#                   'onlyH_act', 'modLFP',mode = mode, inv_exclude = True,
#                    score='balanced_accuracy')
        impr_wrt_base_per_medcond_per_pgn=computeImprovementsPerParcelGroup2(outputs_filtered, 
                 base_perf_prefix = base_perf_prefix, base_perf_low_prefix = base_perf_low_prefix,
                 mode = mode, inv_exclude = True,
                 score = 'balanced_accuracy',
                 ignore_base_prefix_missing=1)
    #        return impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond

        for medcond in impr_wrt_base_per_medcond_per_pgn.keys():
            if len(impr_wrt_base_per_medcond_per_pgn[medcond] ) <= 5:
                print(f'WARNING: {rn} {medcond} has zero len impr_wrt_base_per_medcond_per_pgn[{medcond}]')
                continue
            axs,crlb, info = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                    head_subj_ind=None, mode=mode, subdir=subdir, 
                    savefile_prefix=f'EXPORT_brain_map_area_strength_{rn}_',
                    save_only = not make_plots)

            ress += [info]
            plt.close()
            #break
        #break
resgood = [a is not None for a in ress]
import gc; gc.collect()

In [ ]:
impr_wrt_base_per_medcond_per_pgn

In [ ]:
mode = 'LFPand_only'

In [ ]:
# impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
#     computeImprovementsPerParcelGroup(outputs_filtered, 'onlyH_act', 'modLFP',
#                                       mode = mode, inv_exclude = True)

In [ ]:
impr_per_medcond_per_pgn

In [ ]:
pp.printDict(outputs_filtered,depth_cur=2)

In [ ]:
outputs_filtered.keys()

In [ ]:
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
[ (ii,tpl[:-1], len(tpll[ii][-1]['feature_names_filtered']) ) for ii,tpl in enumerate(tpll ) ]

In [ ]:
ii = 25
print(tpll[ii][:-1])
len(tpll[ii][-1]['feature_names_filtered'])

In [ ]:
len(tpll[ii][-1]['featnames_for_fit'])

In [ ]:
tpll[ii][-1]['cmd']

In [ ]:
tpll[ii][-1].keys()

# Main feature signif plots

In [ ]:
prefixes

In [ ]:
import utils_postprocess as pp
from utils_postprocess_HPC import loadFullScores 
from matplotlib.lines import Line2D


#%debug
hh_ = None
#for prefix_cur in list(sorted( set(prefixes) - set(['all']) )):
#for prefix_cur in ['cross_freqmod_beta,gamma:HFO']:
#for prefix_cur in [ 'LFPrel_noself']:
#for prefix_cur in [ 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyCon']:
#for prefix_cur in [ 'allb_beta', 'onlyH']:
#for prefix_cur in [  'allb_beta', 'allb_tremor', 'allb_gamma', 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyRbcorr', 'LFPrel_noself_onlyCon', 'modSrc_self',  'onlyCBSrc', 'onlyMotorSrc']:
#for prefix_cur in [ 'LFPrel_noself_onlyBpcorr', 'LFPrel_noself']:
#for prefix_cur in [ 'all']:
#for tpl_ in [('onlyH',10), ('LFPrel_noself',25)]:
#for tpl_ in [('LFPrel_noself_onlyRbcorr',15)]:
#for tpl_ in [ ('allb_beta',15),  ('allb_gamma',15), ('allb_tremor',15) ]:
#for tpl_ in [ ('LFPrel_noself_onlyBpcorr',25)]:    
#for prefix_cur in [ 'onlyH', 'LFPrel_noself', 'allb_beta']:
#for prefix_cur in prefixes:

pref_hh_tuples = [ ('all',40)]
pref_hh_tuples = [ ('onlyH',10)]
pref_hh_tuples = [('LFPrel_noself',35) ]

pref_hh_tuples = [ ('onlyH_act',40)]
feat_subset_name = 'all'

subdir_short = 'per_subj_per_medcond_best_LFP'
grpit_tpl = 'merge_movements','trem_vs_hold&move'
#grpit_tpl = 'merge_movements','basic'

#subdir_short = 'joint2_noskip'

from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_HPC.py'
ipython.magic(runstr_)
#             break+
#         break
#     break


# EBM

In [ ]:
prefixes

In [ ]:
aa = [   'onlyH',
 'onlyH_act',
 'onlyH_act_CB',
 'onlyH_act_CByFS',
 'onlyH_act_CByFSyPS',
 'onlyH_act_CByFSyPSyPI',
 'onlyH_act_CByFSyPSyPIyFM',
 'onlyH_act_CBySM',
 'onlyH_act_noCB',
 'onlyH_act_noCBnoFS',
 'onlyH_act_noCBnoFSnoPS',
 'onlyH_act_noCBnoFSnoPSnoPI',
 'onlyH_act_noCBnoFSnoPSnoPInoFM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOI',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoA',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTI',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSM',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSMnoTS',
 'onlyH_act_noCBnoFSnoPSnoPInoFMnoOInoOMnoAnoTInoSMnoTSnoFI',
 'onlyH_act_noFrontalMed',
 'onlyH_act_noFrontalSup',
 'onlyH_act_noLFP',
 'onlyH_act_noOccipitalMid',
 'onlyH_act_noParietalSup',
 'onlyH_act_noSensorimotor',
 'onlyH_act_noTemporalMid',   ]

In [ ]:
aa = [ 
 'allb_beta_noH',
 'allb_gamma_noH',
 'allb_tremor_noH']

In [ ]:
[tpl[2:-1] for tpl in tpll]

In [ ]:
subdir = 'per_subj_per_medcond_best_LFP'
subdir_short = subdir

pref_hh_tuples = [ ('all',40)]
EBM_feat_subsets = ['VIFsel']

pref_hh_tuples = [ ('onlyH_act_SMyOIyFSyFIyTM', 5), ('onlyH',20), ('onlyH_act',20), ('LFPrel_noself',40)]
pref_hh_tuples = [ ('onlyH_act_SMyOIyFSyFIyTM', 5)]
pref_hh_tuples = [ ('LFPrel_noself',40) ]
# pref_hh_tuples = [ ('onlyH_act_noCB',20)]
# pref_hh_tuples = [ ('onlyH_act_noCBnoFS',20)]
# pref_hh_tuples =  [(a,10) for a in aa]
#pref_hh_tuples =  [(a,40) for a in aa]
##pref_hh_tuples = [ ('modSrc',50)]
pref_hh_tuples = [ ('onlyH_act',7)]
#pref_hh_tuples = [ ('modLFP',3)]
#pref_hh_tuples = [ ('onlyH',10)]
#onlyH_act_noCBnoFS
# pref_hh_tuples = [ ('onlyH_act_noFrontalSup',20)]
# pref_hh_tuples = [ ('onlyH_act_noCBnoFSnoPSnoPI',20)]

EBM_feat_subsets = ['all']


#subdir_short = 'joint_noskip'
#grpit_tpl = 'merge_movements','basic'
grpit_tpl = 'merge_movements','trem_vs_hold&move'
#grpit_tpl = 'merge_all_not_trem','basic'

import gc; gc.collect()
#%debug
from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_EBM_HPC.py'
ipython.magic(runstr_)

In [ ]:
int( keystr[1:3] )

In [ ]:
outputs_grouped.keys()

In [ ]:
prefixes

In [ ]:
from datetime import datetime
fname = mult_clf_output['filename_full']
mtime = os.stat(  fname).st_mtime
datetime.fromtimestamp(mtime)

In [ ]:
fname

In [ ]:
mult_clf_output['cmd']

In [ ]:
mult_clf_output_clf_output_clf_output_clf_output     

In [ ]:
d = ['Cerebellum','ParietalSup','FrontalSup', 'ParietalInf','Angular', 'OccipitalInf', 'FrontalMed']
labels_dict['all_raw']

In [ ]:
clf_dict = mult_clf_output['featsel_per_method']['interpret_EBM']['all']
print(clf_dict.keys())

from interpret import preserve
explainer = clf_dict['explainer']
scores = clf_dict['scores']

In [ ]:
sc_noi, fns_noi, _,_ = postp.splitScoresEBM(scores,explainer.feature_names)

In [ ]:
sc_noi

In [ ]:
fn_to_observe = fns_noi[-1]
print(fn_to_observe)
preserve(explainer, fn_to_observe)

In [ ]:
mult_clf_output['feature_names_filtered']

In [ ]:
dd = 'bias/6.1694/2.4873'
np.sum( np.array(list(dd)) == '/' ) 


In [ ]:
import gc;gc.collect()

In [ ]:
common = set(mult_clf_output['VIF_truncation']['colinds_bad_VIFsel'])
#print(common)
for i,og in enumerate(outputs_grouped.items() ):
    rn = og[0][0]
    (prefix,grp,int_type), mult_clf_output = og[1]
    VIFtr = mult_clf_output['VIF_truncation']
    badinds = VIFtr['colinds_bad_VIFsel']
    print(len(badinds), badinds)
    common = common & set(badinds)
    
print(len(common),sorted(common,reverse=1) )

In [ ]:
d[(rn,prefix,grp,it)]

In [ ]:
VIFtr.keys()

In [ ]:
VIFtr['VIFsel_linreg_objs'][0].__dict__.keys()

In [ ]:
d = {}
for rn,prefix,grp,it,mult_clf_output in tpll:
    VIFtr = mult_clf_output['VIF_truncation']
    dd = dict( VIFtr.items() )
    #del dd['VIFsel_linreg_objs']
    #dd[]
    #VIFtr['VIFsel_linreg_objs'][0]
    d[(rn,prefix,grp,it)] = dd
#('S01,S02,S04,S05,S07_off', 'allb_beta', 'merge_movements', 'basic')
VIF_info_fname = pjoin(gv.data_dir,subdir_short,'VIF_info.npz')
print(VIF_info_fname)
np.savez(VIF_info_fname,VIF_info = d)

In [ ]:
# (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
#      scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]


runMLvars_pso = []
for cso in collect_SHAP_outs:
    (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = cso 
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    #assert (prefix,grp,int_type) == (prefix_,grp_,int_type_)    
    cmd = mult_clf_output['cmd']
    print(cmd[0])

    ####################

    s = '%run -i ../run/run_ML.py '
    for a,b in cmd[0]:
        s += f'{a} {b} '
    print(s)
    from IPython import get_ipython; ipython = get_ipython()

    #%run -i ../run/run_ML.py -r S01_on_hold,S01_on_move,S02_on_hold,S02_on_move,S04_on_hold,S04_on_move,S05_on_hold,S05_on_move,S07_on_hold,S07_on_move --param_file ML_joint_one_LFP_HPC.ini --groupings_to_use merge_movements --int_types_to_use basic --feat_types H_act,H_mob,H_compl --prefix onlyH --SLURM_job_id 114703_22 --calc_MI 0

    ipython.magic(s + ' --exit_after artif_processed --show_plots 0 --do_cleanup 0')
    anndict_per_intcat_per_rawn_out = anndict_per_intcat_per_rawn

    runMLvars = {}
    runMLvars['featnames_nice'] = featnames_nice
    runMLvars['featnames'] = featnames
    runMLvars['X_pri'] = X_pri
    runMLvars['rawnames'] = rawnames
    runMLvars['Xtimes_pri'] = Xtimes_pri
    runMLvars['wbd_pri'] = wbd_pri
    runMLvars['anndict_per_intcat_per_rawn'] = anndict_per_intcat_per_rawn
    runMLvars['new_main_side_pri'] = new_main_side_pri
    runMLvars['roi_labels'] = roi_labels
    runMLvars['srcgrouping_names_sorted'] = srcgrouping_names_sorted
    runMLvars['rawtimes_pri'] = rawtimes_pri
    
    runMLvars_pso += [runMLvars]

    rng = slice(0,10)
    
    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    featis = featis[rng] #np.random.randint( len(featnames),size=20)

    # anns, anns_pri, times_concat, dataset_bounds, wbd_merged = utsne.concatAnns(rawnames,
    #                                                           rawtimes_pri, crop=(crop_start,crop_end),
    #                                                           side_rev_pri = side_switch_happened_pri,
    #                                                          wbd_pri = wbd_pri, sfreq=sfreq, ret_wbd_merged=1)
    # print('times_concat end {} wbd end {}'.format(times_concat[-1] * sfreq, wbd_merged[1,-1] ) )

    # ivalis = utils.ann2ivalDict(anns)
    # ivalis_tb_indarrays_merged = \
    #     utils.getWindowIndicesFromIntervals(wbd_merged,ivalis,
    #                                         sfreq,ret_type='bins_contig',
    #                                         ret_indices_type = 'window_inds',
    #                                         nbins_total=nbins_total )

    # anndict_per_intcat = f['anndict_per_intcat'][()]
    # anndict_per_intcat_per_rawn[rawn] = anndict_per_intcat
    # bindict_per_rawn[rawn] = upre.markedIntervals2Bins(anndict_per_intcat,
    #                                                    rawtimes,sfreq,wbd=wbd)
    xlim = (-4,8)
    #xlim = None
    figname = 'hists_'+ f'{(rn_,prefix,grp,int_type,label_str,fsh)}_rng={rng}.pdf'
    print('start plotting ',figname)
    postp.plotFeatHists(rawnames,featnames_nice,featis,X_pri,Xconcat, bindict_per_rawn,
                      ivalis_tb_indarrays_merged, xlim_common = xlim,nbins=25, savefig = 0)
    
    
    plt.savefig(pjoin(gv.dir_fig, subdir_short, figname))
    plt.close()


In [ ]:
import gc; gc.collect()

In [ ]:
(prefix_,grp_,int_type_)

## get info from run_genfeats (for a fixed rn_)

In [ ]:
mainLFPchan_per_rawn = {}
for runMLvars in runMLvars_pso:
    for rawn in runMLvars['rawnames']:
        subj,medcond,task  = utils.getParamsFromRawname(rawn)
        mainLFPchan_cur = best_LFP_info[subj][f'{best_LFP_prefix},merge_movements,basic']['best_LFP']
        #print(rawn,subj,mainLFPchan_cur)
        mainLFPchan_per_rawn[rawn] = mainLFPchan_cur

In [ ]:



rns = list(output_per_raw.keys() )
genfeat_info_per_rn = {}
for rn_to_plot in rns:
    mult_clf_output = output_per_raw[rn_][prefix_][grp][int_type]
    ML_info = mult_clf_output['info']
    fn_feat = ML_info['fname_feat_full_pri']

    dat_out_pri = []
    dat_lfp_hires_out_pri = []
    times_hires_out_pri = []
    times_out_pri = []
    subfeature_order_out_pri = []
    subfeature_order_lfp_hires_out_pri = []
    extdat_out_pri = []
    rawnames_out = []
    emgdat_pri = []
    
    for i,fn_feat_cur in enumerate(fn_feat):
        ffeat = np.load(fn_feat_cur, allow_pickle=True )
        ss = ' '.join([' '.join(list(tpl)) for tpl in  ffeat['cmd'][0]] )
        s = '%run -i ../run/run_genfeats.py ' + ss + ' --exit_after prescale_data'
        print(s)
        ipython.magic(s)

        dat_out_pri              += dat_pri
        dat_lfp_hires_out_pri    += dat_lfp_hires_pri
        times_hires_out_pri      += times_hires_pri
        times_out_pri            += times_pri
        subfeature_order_out_pri += subfeature_order_pri
        subfeature_order_lfp_hires_out_pri += subfeature_order_lfp_hires_pri
        extdat_out_pri           += extdat_pri
    
    import re
    for fn in fn_feat:
        r = re.match('.*(S[0-9]+_[a-z]+_[a-z]+)_.*',fn).groups()[0]
        #rint(r)
        rawnames_out += [r]


    emgraw_pri = []
    for rawi,rawn in enumerate(ML_info['rawnames']):
        emgraw = mne.io.read_raw_fif ( pjoin(gv.data_dir,rawn + '_emg_rectconv.fif') )
        chns_cur = gv.EMG_per_hand[main_side_pri[rawi]]
        emgraw.pick_channels(chns_cur)
        emgraw.resample(256)
        #emgraw_pri += [emgraw]
        emgdat_pri += [emgraw.get_data()]
        assert emgdat_pri[-1].shape[1] == len(times_out_pri[rawi])

    from featlist import replaceMEGsrcChnamesParams
    subfeature_order_newsrcgrp_pri = ML_info.get('chnames_newsrcgrp_pri')
    if subfeature_order_newsrcgrp_pri is None:
        subfeature_order_newsrcgrp_pri = [0] * len(rawnames_out)
        for rawi in range(len(runMLvars['rawnames'])):
            subfeature_order_newsrcgrp_pri[rawi] = \
                replaceMEGsrcChnamesParams(subfeature_order_out_pri[rawi], 0,9, '.*', 0)
            
            
    d = {}
    d['dat_out_pri'] = dat_out_pri
    d['dat_lfp_hires_out_pri'] = dat_lfp_hires_out_pri
    d['times_hires_out_pri'] = times_hires_out_pri
    d['times_out_pri'] = times_out_pri
    d['subfeature_order_out_pri'] = subfeature_order_out_pri
    d['subfeature_order_lfp_hires_out_pri'] = subfeature_order_lfp_hires_out_pri
    d['extdat_out_pri'] = extdat_out_pri
    d['rawnames_out'] = rawnames_out
    d['emgdat_pri'] = emgdat_pri
    d['subfeature_order_newsrcgrp_pri'] = subfeature_order_newsrcgrp_pri
    genfeat_info_per_rn[rn_to_plot] = d

# best_LFP_info_fname = pjoin(gv.data_dir, 'best_LFP_info.json')
# with open(best_LFP_info_fname, 'r') as f:
#     best_LFP_info = json.load(f)



display(mainLFPchan_per_rawn)

In [ ]:
rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats

In [ ]:
# plot time traces

for collect_SHAP_outs_ind in range(len(collect_SHAP_outs)):
    #collect_SHAP_outs_ind = 0
    (rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[collect_SHAP_outs_ind]
    print(rn_)
    
    runMLvars = runMLvars_pso[collect_SHAP_outs_ind]
    d = genfeat_info_per_rn[rn_]
    
#     ##########################  just for now to avoid recalc of genfeats
#     if rn_.endswith('_off'):
#         continue
#     ###########################    
        
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    VIF_truncation = mult_clf_output['VIF_truncation']
    colinds_good_VIFsel = VIF_truncation['colinds_good_VIFsel']
    featnames_VIFsel = runMLvars['featnames'][colinds_good_VIFsel]
    featnames_best_LFP = runMLvars['featnames']

    from globvars import gp
    beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    #%debug
    rng = slice(0,8)
    #rng = slice(0,2)
    rng_feats = slice(0,2)
    #rng_feats = slice(0,1)
    from plots import plotFeatsAndRelDat
    
    axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                             featnames_VIFsel[featis[rng_feats]], d['dat_out_pri'],
                       d['subfeature_order_out_pri'],
                    runMLvars['X_pri'],[featnames_best_LFP]*len(d['rawnames_out']),
                    d['times_out_pri'],
                    runMLvars['Xtimes_pri'],
                    d['subfeature_order_newsrcgrp_pri'], runMLvars['wbd_pri'],
                    dat_hires_pri=d['dat_lfp_hires_out_pri'],
                    chnames_all_hires_pri = d['subfeature_order_lfp_hires_out_pri'],
                    times_hires_pri=d['times_hires_out_pri'],
                    anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                    sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                    roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                            main_side_let = main_side_let, ww=6,hh=3,
                            beh_states_to_shade = beh_states_to_shade, 
                             extdat_pri=d['emgdat_pri'])

    plt.tight_layout()
    plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,label_str,fsh)}_{rng}.pdf'))
    plt.close()

In [ ]:
len(collect_SHAP_outs)

# interpretML

In [ ]:
#for prefix_cur in prefixes:
#for prefix_cur in prefixes[:3]:
#for prefix_cur in prefixes:
for prefix in ['onlyMotorSrc']:
    prefixes_to_use = [prefix_cur]

    outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,
                                      ['merge_movements'],['trem_vs_hold&move'])
    print('      ',prefix_cur)
    for og in outputs_grouped.items():
        rn = og[0]
        (prefix,grp,int_type), mult_clf_output = og[1]
        assert prefix == prefix_cur
    
        filename_fullsize = mult_clf_output['filename_full']
        from pathlib import Path
        pfsz = Path(filename_fullsize)
        filename_fullsize = pjoin(pfsz.parents[0], pfsz.name[2:])
        #    finfo = os.stat( filename_fullsize )
        #    print(finfo.st_size / (1024**2))
        f = np.load(filename_fullsize,allow_pickle=True)

        results_cur =  f['results_cur'][()]

        #results_cur.keys()

        EBM = results_cur['featsel_per_method']['interpret_EBM']        
        #scores

        clf_dict = EBM['info_per_cp'][('trem_L', 'hold_L&move_L')]
        scores = clf_dict['scores']; 
        #print(len(results_cur['feature_names_filtered']), len(EBM['feature_indices_used']) )
        #print('len(scores) = ',len(scores) )
        explainer = clf_dict['explainer']
        
        print((rn,grp,int_type), utsne.sprintfPerfs(clf_dict['perf'] ) )

        del f
        del results_cur
        gc.collect()

In [ ]:
#       LFPrel_noself
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.70%,97.30%,73.97%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 54.75%,93.76%,65.06%
#        LFPrel_noself_onlyBpcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.09%,96.59%,64.78%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 49.71%,93.22%,60.30%
#        LFPrel_noself_onlyCon
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 42.19%,94.97%,53.35%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 30.51%,92.18%,41.29%
#        LFPrel_noself_onlyRbcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 48.94%,97.15%,62.01%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.56%,92.61%,46.82%
#        allb_beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 64.64%,95.34%,72.04%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.94%,90.69%,63.34%
#        cross_freqmod_beta,gamma:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 18.93%,97.11%,29.55%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 26.17%,94.16%,37.62%
#        cross_freqmod_tremor,beta:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 40.86%,95.38%,52.56%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.36%,92.31%,46.41%
#        cross_freqmod_tremor,gamma:beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 39.69%,96.07%,52.15%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 27.67%,94.66%,39.64%
#        modLFP
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 35.31%,95.57%,47.26%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 4.50%,98.84%,8.41%
#        modSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 72.72%,97.23%,80.13%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 62.01%,93.29%,70.18%
#        modSrc_self
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.11%,95.55%,71.22%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.49%,90.39%,62.74%
#        onlyCBSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.38%,95.51%,63.69%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 22.85%,94.62%,33.94%
#        onlyH
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 62.04%,95.14%,69.86%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 51.43%,91.03%,60.01%
#        onlyMotorSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 74.52%,97.34%,81.46%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 63.63%,94.94%,72.74%
#        onlyRestSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 66.35%,96.35%,74.51%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 60.19%,92.75%,68.41%

In [ ]:
(rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
     scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]

print(rn_,prefix,grp,int_type)
mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_off']['onlyH'][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_on']['onlyMotorSrc'][grp][int_type]

In [ ]:
print(range(0,6))

In [ ]:
[4  for d in [4,5] if d in [4,5]]

In [ ]:
help(plotFeatsAndRelDat)

In [ ]:
del plotFeatsAndRelDat

In [ ]:
import plots
import importlib as il; il.reload(plots)

In [ ]:
runMLvars['wbd_pri'][0]

In [ ]:
extdat_out_pri[0].shape

In [ ]:
from globvars import gp
beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

# TODO: right now these are not correct scores (perhaps) because of the way they are averaged
#%debug
rng = slice(0,8)
#rng = slice(0,2)
rng_feats = slice(0,2)
#rng_feats = slice(0,1)
from plots import plotFeatsAndRelDat
axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                         featnames_VIFsel[featis[rng_feats]], dat_out_pri,
                   subfeature_order_out_pri,
                runMLvars['X_pri'],[featnames_best_LFP]*len(rawnames_out),
                times_out_pri,
                runMLvars['Xtimes_pri'],
                subfeature_order_newsrcgrp_pri, runMLvars['wbd_pri'],
                dat_hires_pri=dat_lfp_hires_out_pri,
                chnames_all_hires_pri = subfeature_order_lfp_hires_out_pri,
                times_hires_pri=times_hires_out_pri,
                anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                        main_side_let = main_side_let, ww=6,hh=3,
                        beh_states_to_shade = beh_states_to_shade, 
                         extdat_pri=emgdat_pri)

plt.tight_layout()
plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,fsh)}_{rng}.pdf'))
plt.close()

In [ ]:
best_feat_name

In [ ]:
print( utsne.sprintfPerfs(clf_dict['perf'] ) )
print( clf_dict['confmat_normalized'] )

In [ ]:
np.argmax(scores)

In [ ]:
assert len( explainer.feature_names ) == len(scores)
best_feat_name = explainer.feature_names [ np.argmax(scores)]
print(best_feat_name)
sortinds = np.argsort(scores)
last_ind = np.where([ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in  sortinds])[0][-1]
last_ind = sortinds[last_ind]
best_feat_name_no_int = explainer.feature_names[last_ind]
print(best_feat_name_no_int)

In [ ]:
sortinds[-1]

In [ ]:
 len(scores), last_ind

In [ ]:
explainer.feature_names[last_ind], m[last_ind], best_feat_name_no_int.find(' x ')

In [ ]:
np.where(m)[0][-1]

In [ ]:
m = [ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in np.argsort(scores) ]
print(m[344])

In [ ]:
print( explainer.__dict__.keys() )
# print( explainer.explanation_type, explainer.name, 
#      explainer.selector, explainer.feature_types)

In [ ]:
mult_clf_output['featsel_per_method']['interpret_EBM']['all'].keys()

from interpret import preserve
explainer = clf_dict['explainer']
preserve(explainer, best_feat_name_no_int)

In [ ]:
preserve(explainer, best_feat_name)

In [ ]:
explainer.visualize()

In [ ]:
# from interpret import show
# show(explainer)  # does not work if open saved

In [ ]:
# Trying to get a better distr fit for Hjorth (no success)

In [ ]:
len(featnames_nice), len(featnames_VIFsel)

In [ ]:
np.array(featnames_nice)[featis[-2:]]

In [ ]:
featnames[featis[-2:]]

In [ ]:
roi_labels['all_raw'][30]

In [ ]:
len(featnames_nice), len(featnames)

In [ ]:
featnames_nice[-5:], featnames[-5:]

In [ ]:
pjoin(subdir,'good_feats_dyn.pdf')

In [ ]:
# from plots import plotFeatsWithEverything
# #%debug
# plotFeatsWithEverything(dat_pri, rawtimes_pri, X_pri, 
#                             Xtimes_pri, dat_lfp_hires_pri, times_hires_pri,
#                             rawnames,
#                             subfeature_order_pri, subfeature_order_newsrcgrp_pri,
#                             subfeature_order_lfp_hires_pri,
#                             anndict_per_intcat_per_rawn,
#                             featnames, wbd_pri,
#                             sfreq, raw_perband_flt_pri, raw_perband_bp_pri,
#                             scale_data_combine_type,
#                             stats_multiband_flt, stats_multiband_bp,
#                             test_plots_descr, special_chns,
#                            fband_names_inc_HFO )

In [ ]:
np.min(Xconcat), np.max(Xconcat_to_fit)

In [ ]:
dd0 = Xconcat[:,featis[0]]
#dd = dd[ dd< np.quantile(dd,0.95)]
dd = np.exp(dd0) 
#dd = 1/dd
dd = np.log( dd - np.min(dd) *.99 )
#dd = dd[ dd< np.quantile(dd,0.99)]
plt.hist(dd,bins=250,alpha=0.5,label='new');

plt.hist(dd0,bins=250,alpha=0.5,label='old');
plt.legend(loc='upper right')

In [ ]:
plt.hist(dd0,bins=250)

In [ ]:
np.min(dd)

In [ ]:
h

In [ ]:
help(dist.pdf)

In [ ]:
f

In [ ]:
import scipy.stats
qm = np.quantile(dd,0.99)
y = dd [ dd < qm]
#y = np.exp(y)
size = len(y)
x = np.arange(np.min(y),np.max(y),0.5)
#y = 0.01 -np.min(dd) + dd
#x = np.arange(0.01,55,0.5)
#y = scipy.int_(np.round_(scipy.stats.vonmises.rvs(5,size=size)*47))
h = plt.hist(y, bins=x)
binh,binloc,barcont = h

# beta, rayleigh look ok
#ten best
# dist_names = ['cauchy',
#  'dweibull',
#  'foldcauchy',
#  'dgamma',
#  't',
#  'tukeylambda',
#  'fisk',
#  'weibull_min', 'erlang']

#'loglaplace',
 
dist_names = ['alpha', 'cauchy',  'dgamma', 'dweibull', 'erlang', 'exponweib',
              'genlogistic',  'foldcauchy',  'genextreme']
#  'fisk',  'fatiguelife',  'gumbel_r',  'gengamma','invweibull',
#  'invgamma',  'mielke',  'powerlognorm',  'nct',
#  'johnsonsu',  'lognorm',  'invgauss',  'loglaplace',
#  'recipinvgauss',  'pearson3',  'laplace', 't',  'tukeylambda',
#  'hypsecant',  'weibull_min',  'logistic',  'kstwobign',
#               'nakagami',  'rayleigh',  'rice',  'maxwell']
dist_names = ['beta']
dist_names = ['levy_stable']

do_plot = 1
d_per_name = {}
pdf_fit_per_name = {}

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    params = dist.fit(y)
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]
    if arg:
        pdf_fitted = dist.pdf(binloc, *arg, loc=loc, scale=scale) * size
    else:
        pdf_fitted = dist.pdf(binloc, loc=loc, scale=scale) * size
    if do_plot and np.max(pdf_fitted) < np.max(binh) * 3:
        plt.plot(binloc, pdf_fitted, label=dist_name)
    plt.xlim(-6,20)
    pdf_fit_per_name[dist_name] = pdf_fitted
    
    discrep = np.sum(np.abs( binh-pdf_fitted[:-1] ) ) / len(binh)
    d_per_name[dist_name] = discrep
plt.legend(loc='upper right')
plt.show()

In [ ]:
arg

In [ ]:
help(sorted)

In [ ]:
stl = sorted( list( d_per_name.items() ), key = lambda x: x[1] )
stl_ext = [ (dn,v,pdf_fit_per_name[dn][0]) for  (dn, v) in stl]
display(stl_ext)

In [ ]:
list( set( ['exponpow', 'f', 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 
               'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 
               'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 
               'gilbrat', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy'] ) - \
set( ['frechet_r',
 'genhalflogistic',
 'foldnorm',
 'gilbrat',
 'gompertz',
 'genpareto',
 'genexpon',
 'halfcauchy',
 'exponpow',
 'gamma',
 'frechet_l',
 'f',
 'gausshyper',
 'gumbel_l'] ) )

In [ ]:
list(list(zip(*stl))[0])[:10]

In [ ]:
list(list(zip(*stl))[0])[-14:]

In [ ]:
list(list(zip(*stl))[0])[-10:]

In [ ]:
len(binh), len(pdf_fitted)

In [ ]:
plt.plot(binh)
plt.plot(pdf_fitted[:-1])

In [ ]:
help(plt.subplots_adjust)